# Deep Learning for Air Quality Analysis: PM2.5 Forecasting, Data Repair & Anomaly Detection

## AI Data Architect Implementation

This notebook implements a comprehensive deep learning system for air quality analysis with the following capabilities:

### 1. Learning and Data Repair
- Ingest 30+ days of PM2.5 data from Air4Thai API
- Train LSTM (Long Short-Term Memory) neural network to learn temporal patterns
- Forecast future PM2.5 values
- Repair missing/incomplete data using learned patterns

### 2. Data Verification and Anomaly Detection
- Implement autoencoder-based anomaly detection
- Detect and flag anomalous data points
- Analyze potential causes of irregularities
- Provide comprehensive reporting

---

In [ ]:
# install required packages with !pip for this project
!pip install numpy pandas matplotlib seaborn scikit-learn requests beautifulsoup4 nltk tensorflow


## Setup and Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 1. Data Ingestion

Fetch PM2.5 data from Air4Thai API for the specified date range (minimum 30 days).

In [ ]:
class AirQualityDataFetcher:
    """
    Fetches and preprocesses air quality data from Air4Thai API
    """
    
    def __init__(self, station_id='36t', param='PM25'):
        self.station_id = station_id
        self.param = param
        self.base_url = "http://air4thai.com/forweb/getHistoryData.php"
    
    def fetch_data(self, start_date, end_date):
        """
        Fetch data from Air4Thai API
        
        Args:
            start_date: Start date in format 'YYYY-MM-DD'
            end_date: End date in format 'YYYY-MM-DD'
        
        Returns:
            DataFrame with hourly PM2.5 measurements
        """
        url = f"{self.base_url}?stationID={self.station_id}&param={self.param}&type=hr&sdate={start_date}&edate={end_date}&stime=00&etime=23"
        
        print(f"Fetching data from: {url}")
        print(f"Date range: {start_date} to {end_date}")
        
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            if 'stations' in data and len(data['stations']) > 0:
                station_data = data['stations'][0]
                station_name = station_data.get('stationNameE', 'Unknown')
                print(f"Station: {station_name}")
                
                # Extract data points
                measurements = station_data.get('data', [])
                df = pd.DataFrame(measurements)
                print(f"Retrieved {len(df)} data points")
                
                return df
            else:
                print("No station data found in response")
                return pd.DataFrame()
                
        except Exception as e:
            print(f"Error fetching data: {e}")
            return pd.DataFrame()
    
    def preprocess_data(self, df):
        """
        Preprocess raw data: parse timestamps, handle missing values, create full time index
        """
        if df.empty:
            return df
        
        # Parse timestamp
        df['DATETIMEDATA'] = pd.to_datetime(df['DATETIMEDATA'])
        df.set_index('DATETIMEDATA', inplace=True)
        
        # Convert PM25 to numeric, marking invalid values as NaN
        df['PM25'] = pd.to_numeric(df['PM25'], errors='coerce')
        
        # Create complete hourly index
        full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
        df = df.reindex(full_index)
        
        # Store original data for comparison
        df['PM25_original'] = df['PM25'].copy()
        
        # Mark missing data
        df['is_missing'] = df['PM25'].isna()
        
        print(f"\nData Summary:")
        print(f"Total hours: {len(df)}")
        print(f"Missing values: {df['is_missing'].sum()} ({df['is_missing'].sum()/len(df)*100:.2f}%)")
        print(f"Valid values: {(~df['is_missing']).sum()}")
        
        return df

# Initialize fetcher
fetcher = AirQualityDataFetcher(station_id='36t')

# Fetch data (30+ days)
start_date = '2025-11-02'
end_date = '2025-12-01'

raw_data = fetcher.fetch_data(start_date, end_date)
df = fetcher.preprocess_data(raw_data)

# Display first few rows
print("\nFirst 10 rows:")
df.head(10)

## 2. Exploratory Data Analysis

In [ ]:
# Statistical summary
print("Statistical Summary:")
print(df['PM25'].describe())

# Visualize raw data with missing values highlighted
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Time series with missing values
axes[0].plot(df.index, df['PM25'], label='PM2.5 Values', color='blue', linewidth=1.5)
missing_points = df[df['is_missing']]
axes[0].scatter(missing_points.index, [df['PM25'].mean()] * len(missing_points), 
                color='red', s=50, alpha=0.6, label=f'Missing Data Points ({len(missing_points)})', 
                marker='x')
axes[0].set_title('PM2.5 Time Series - Raw Data with Missing Values Highlighted', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Distribution
axes[1].hist(df['PM25'].dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[1].axvline(df['PM25'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["PM25"].mean():.2f}')
axes[1].axvline(df['PM25'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["PM25"].median():.2f}')
axes[1].set_title('PM2.5 Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('PM2.5 (µg/m³)')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze missing data patterns
print(f"\nMissing Data Analysis:")
print(f"Longest consecutive missing period: {df['is_missing'].astype(int).groupby((df['is_missing'] != df['is_missing'].shift()).cumsum()).sum().max()} hours")

## 3. Data Preparation for Deep Learning

Create sequences for LSTM training and prepare features.

In [ ]:
class DataPreparator:
    """
    Prepares time series data for deep learning models
    """
    
    def __init__(self, sequence_length=24):
        self.sequence_length = sequence_length
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        
    def create_features(self, df):
        """
        Engineer features from time series data
        """
        df = df.copy()
        
        # Temporal features
        df['hour'] = df.index.hour
        df['day_of_week'] = df.index.dayofweek
        df['day_of_month'] = df.index.day
        df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
        
        # Cyclical encoding for hour (24-hour cycle)
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        
        # Cyclical encoding for day of week (7-day cycle)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        
        return df
    
    def create_sequences(self, data, target_col='PM25', feature_cols=None):
        """
        Create sequences for LSTM training
        
        Args:
            data: DataFrame with features
            target_col: Column name for target variable
            feature_cols: List of feature column names (if None, uses all numeric columns)
        
        Returns:
            X: Input sequences
            y: Target values
            mask: Boolean array indicating if target value was originally missing
        """
        if feature_cols is None:
            feature_cols = ['PM25', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend']
        
        # For training, use only non-missing data initially
        # We'll fill missing values with forward/backward fill temporarily
        df_filled = data.copy()
        df_filled['PM25_filled'] = df_filled['PM25'].fillna(method='ffill').fillna(method='bfill')
        
        # Scale the data
        feature_data = df_filled[feature_cols].copy()
        feature_data['PM25'] = df_filled['PM25_filled']
        
        scaled_data = self.scaler.fit_transform(feature_data)
        
        X, y, masks = [], [], []
        
        for i in range(len(scaled_data) - self.sequence_length):
            X.append(scaled_data[i:i + self.sequence_length])
            y.append(scaled_data[i + self.sequence_length, 0])  # PM25 is first column
            # Check if the target value was originally missing
            masks.append(data.iloc[i + self.sequence_length]['is_missing'])
        
        return np.array(X), np.array(y), np.array(masks)
    
    def inverse_transform_pm25(self, scaled_values):
        """
        Inverse transform PM2.5 values from scaled to original range
        """
        # Create dummy array with same shape as scaler input
        dummy = np.zeros((len(scaled_values), self.scaler.n_features_in_))
        dummy[:, 0] = scaled_values.flatten()
        inversed = self.scaler.inverse_transform(dummy)
        return inversed[:, 0]

# Prepare data
preparator = DataPreparator(sequence_length=24)  # Use 24 hours (1 day) of history
df_featured = preparator.create_features(df)

print("Features created:")
print(df_featured.columns.tolist())

# Create sequences
X, y, masks = preparator.create_sequences(df_featured)

print(f"\nSequence shapes:")
print(f"X shape: {X.shape} (samples, sequence_length, features)")
print(f"y shape: {y.shape}")
print(f"Missing target values: {masks.sum()} out of {len(masks)}")

## 4. LSTM Model for Forecasting and Data Repair

Build and train an LSTM neural network to:
1. Learn temporal patterns in PM2.5 data
2. Forecast future values
3. Repair missing/incomplete data

In [ ]:
class LSTMForecaster:
    """
    LSTM-based forecasting model for PM2.5 prediction and data repair
    """
    
    def __init__(self, sequence_length=24, n_features=6):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.model = None
        self.history = None
    
    def build_model(self):
        """
        Build LSTM architecture
        """
        model = Sequential([
            # First LSTM layer with return sequences
            LSTM(128, activation='relu', return_sequences=True, 
                 input_shape=(self.sequence_length, self.n_features)),
            Dropout(0.2),
            
            # Second LSTM layer
            LSTM(64, activation='relu', return_sequences=True),
            Dropout(0.2),
            
            # Third LSTM layer
            LSTM(32, activation='relu'),
            Dropout(0.2),
            
            # Dense layers
            Dense(16, activation='relu'),
            Dense(1)  # Output: single PM2.5 value
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae', 'mse']
        )
        
        self.model = model
        return model
    
    def train(self, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
        """
        Train the LSTM model
        """
        # Callbacks
        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )
        
        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=0.00001
        )
        
        print("Training LSTM model...")
        self.history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
        
        return self.history
    
    def predict(self, X):
        """
        Make predictions
        """
        return self.model.predict(X, verbose=0)
    
    def plot_training_history(self):
        """
        Visualize training progress
        """
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss
        axes[0].plot(self.history.history['loss'], label='Training Loss', linewidth=2)
        axes[0].plot(self.history.history['val_loss'], label='Validation Loss', linewidth=2)
        axes[0].set_title('Model Loss During Training', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss (MSE)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # MAE
        axes[1].plot(self.history.history['mae'], label='Training MAE', linewidth=2)
        axes[1].plot(self.history.history['val_mae'], label='Validation MAE', linewidth=2)
        axes[1].set_title('Model MAE During Training', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('MAE')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Split data into train and validation sets
# Use only non-missing targets for training
non_missing_idx = ~masks
X_clean = X[non_missing_idx]
y_clean = y[non_missing_idx]

X_train, X_val, y_train, y_val = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42, shuffle=False
)

print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")

# Build and train model
forecaster = LSTMForecaster(sequence_length=24, n_features=X.shape[2])
forecaster.build_model()

print("\nModel Architecture:")
forecaster.model.summary()

In [ ]:
# Train the model
history = forecaster.train(X_train, y_train, X_val, y_val, epochs=100, batch_size=32)

# Plot training history
forecaster.plot_training_history()

## 5. Model Evaluation and Data Repair

Evaluate the model and use it to repair missing data.

In [ ]:
# Make predictions on all data
y_pred_scaled = forecaster.predict(X)

# Inverse transform to get actual PM2.5 values
y_pred = preparator.inverse_transform_pm25(y_pred_scaled)
y_true = preparator.inverse_transform_pm25(y)

# Evaluate on validation set
val_idx = len(X_train)
y_val_pred = y_pred[val_idx:val_idx + len(X_val)]
y_val_true = y_true[val_idx:val_idx + len(X_val)]

mse = mean_squared_error(y_val_true, y_val_pred)
mae = mean_absolute_error(y_val_true, y_val_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_true, y_val_pred)

print("Validation Set Performance:")
print(f"MAE: {mae:.3f} µg/m³")
print(f"RMSE: {rmse:.3f} µg/m³")
print(f"R² Score: {r2:.3f}")

# Repair missing data
df_repaired = df_featured.copy()

# Align predictions with dataframe (skip first sequence_length rows)
df_repaired.loc[df_repaired.index[preparator.sequence_length:], 'PM25_predicted'] = y_pred

# Use predictions to fill missing values
df_repaired['PM25_repaired'] = df_repaired['PM25'].copy()
missing_mask = df_repaired['is_missing'] & df_repaired['PM25_predicted'].notna()
df_repaired.loc[missing_mask, 'PM25_repaired'] = df_repaired.loc[missing_mask, 'PM25_predicted']

# Fill any remaining gaps at the start
df_repaired['PM25_repaired'] = df_repaired['PM25_repaired'].fillna(method='bfill')

repaired_count = missing_mask.sum()
print(f"\nData Repair:")
print(f"Missing values repaired: {repaired_count}")
print(f"Remaining missing values: {df_repaired['PM25_repaired'].isna().sum()}")

In [ ]:
# Visualize predictions vs actual values
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Plot 1: Validation predictions vs actual
val_dates = df_featured.index[preparator.sequence_length + val_idx:preparator.sequence_length + val_idx + len(X_val)]
axes[0].plot(val_dates, y_val_true, label='Actual PM2.5', linewidth=2, alpha=0.7)
axes[0].plot(val_dates, y_val_pred, label='Predicted PM2.5', linewidth=2, alpha=0.7)
axes[0].set_title('BASELINE LSTM Model: Predictions vs Actual (Validation Set)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Scatter plot of predictions vs actual
axes[1].scatter(y_val_true, y_val_pred, alpha=0.5)
axes[1].plot([y_val_true.min(), y_val_true.max()], 
             [y_val_true.min(), y_val_true.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_title(f'Prediction Accuracy (R² = {r2:.3f})', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Actual PM2.5 (µg/m³)')
axes[1].set_ylabel('Predicted PM2.5 (µg/m³)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Complete time series with repaired data
axes[2].plot(df_repaired.index, df_repaired['PM25'], label='Original Data', 
             linewidth=1.5, alpha=0.6, color='blue')
axes[2].plot(df_repaired.index, df_repaired['PM25_repaired'], label='Repaired Data (Missing values filled)', 
             linewidth=1.5, alpha=0.8, color='green')
repaired_points = df_repaired[df_repaired['is_missing']]
axes[2].scatter(repaired_points.index, repaired_points['PM25_repaired'], 
                color='red', s=50, alpha=0.8, label=f'Repaired Points ({repaired_count})', marker='o', zorder=5)
axes[2].set_title('Complete Time Series with Deep Learning-Based Data Repair', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('PM2.5 (µg/m³)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Store baseline metrics for comparison
baseline_metrics = {
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2,
    'MSE': mse
}

print("\n" + "="*80)
print("BASELINE MODEL PERFORMANCE SUMMARY")
print("="*80)
print(f"MAE:  {mae:.4f} µg/m³")
print(f"RMSE: {rmse:.4f} µg/m³")
print(f"MSE:  {mse:.4f}")
print(f"R²:   {r2:.4f}")
print("="*80)

In [ ]:
# Visualize Results
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Plot 1: Full time series
axes[0].plot(df_quick.index, df_quick['PM25_original'], 
            label='Original Complete Data', linewidth=2, color='blue', alpha=0.5)
axes[0].plot(df_quick.index, df_quick['PM25_damaged'], 
            label='Data with 3 Blocks Removed', linewidth=1.5, color='gray', alpha=0.7, linestyle=':')

# Highlight removed blocks
for block in removed_blocks:
    axes[0].axvspan(block['start'], block['end'], alpha=0.2, color='red', label='Removed Block' if block['block']==1 else '')

axes[0].set_title('BEFORE Repair: 3 Consecutive 24-Hour Blocks Removed', fontsize=14, fontweight='bold')
axes[0].set_ylabel('PM2.5 (µg/m³)', fontsize=11)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: After repair
axes[1].plot(df_quick.index, df_quick['PM25_original'], 
            label='Original (Ground Truth)', linewidth=2, color='blue', alpha=0.5)
axes[1].plot(aligned_idx_quick, repaired_quick, 
            label='LSTM Repaired', linewidth=2, color='green', alpha=0.9)

# Highlight repaired areas
gap_mask_aligned = gaps_quick
repaired_points = aligned_idx_quick[gap_mask_aligned]
repaired_vals = repaired_quick[gap_mask_aligned]
axes[1].scatter(repaired_points, repaired_vals, 
               color='red', s=50, alpha=0.7, label=f'Repaired Points ({len(repaired_points)})',
               marker='o', zorder=5, edgecolors='darkred')

axes[1].set_title('AFTER Repair: Gaps Filled by LSTM', fontsize=14, fontweight='bold')
axes[1].set_ylabel('PM2.5 (µg/m³)', fontsize=11)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Plot 3: Comparison (Actual vs Repaired for gaps)
axes[2].scatter(eval_quick['true_values'], eval_quick['predicted_values'], 
               alpha=0.6, s=70, color='red', edgecolors='black', linewidths=1.5)

min_v = min(eval_quick['true_values'].min(), eval_quick['predicted_values'].min())
max_v = max(eval_quick['true_values'].max(), eval_quick['predicted_values'].max())
axes[2].plot([min_v, max_v], [min_v, max_v], 'b--', linewidth=2.5, label='Perfect Repair')

axes[2].set_title(f'Repair Accuracy: MAE={eval_quick["MAE"]:.2f} µg/m³, R²={eval_quick["R2"]:.3f}', 
                 fontsize=14, fontweight='bold')
axes[2].set_xlabel('Actual PM2.5 (µg/m³)', fontsize=11)
axes[2].set_ylabel('Repaired PM2.5 (µg/m³)', fontsize=11)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "╔" + "="*78 + "╗")
print("║" + " "*25 + "QUICK START SUMMARY" + " "*34 + "║")
print("╠" + "="*78 + "╣")
print(f"║  Removed:    {total_removed} hours in 3 consecutive blocks ({removal_pct:.1f}%)" + " "*(26) + "║")
print(f"║  Repaired:   {eval_quick['n_gaps']} gaps with LSTM" + " "*(50-len(str(eval_quick['n_gaps']))) + "║")
print(f"║  Accuracy:   MAE = {eval_quick['MAE']:.3f} µg/m³" + " "*46 + "║")
print(f"║  Quality:    R² = {eval_quick['R2']:.3f} (explains {eval_quick['R2']*100:.1f}% of variance)" + " "*(25-len(f"{eval_quick['R2']*100:.1f}")) + "║")
print("╠" + "="*78 + "╣")

if eval_quick['MAE'] < 3:
    print("║  ✅ EXCELLENT repair quality - Ready for production use!" + " "*27 + "║")
elif eval_quick['MAE'] < 5:
    print("║  ✅ GOOD repair quality - Suitable for most applications" + " "*24 + "║")
else:
    print("║  ⚠️  MODERATE repair quality - Consider more training data" + " "*23 + "║")

print("╚" + "="*78 + "╝")

# Save results
output_file = 'quick_start_repair_results.csv'
df_quick[['PM25_original', 'PM25_damaged', 'PM25_repaired', 'was_removed']].to_csv(output_file)
print(f"\n💾 Results saved to: {output_file}")

In [ ]:
print("\n" + "="*80)
print("STEP 3: REPAIR GAPS AND EVALUATE")
print("="*80)

# Repair data
print("\n  Applying LSTM to repair gaps...")
repaired_quick = quick_repairer.repair_data(X_quick)

# Evaluate
eval_quick = quick_repairer.evaluate_repair(orig_quick, repaired_quick, gaps_quick)

print(f"\n  ✓ Repaired {eval_quick['n_gaps']} gap points")
print(f"\n  📊 Repair Quality:")
print(f"     MAE:  {eval_quick['MAE']:.3f} µg/m³")
print(f"     RMSE: {eval_quick['RMSE']:.3f} µg/m³")
print(f"     R²:   {eval_quick['R2']:.3f}")

# Add to dataframe
seq_len_quick = quick_repairer.sequence_length
aligned_idx_quick = df_quick.index[seq_len_quick:]
df_quick['PM25_repaired'] = np.nan
df_quick.loc[aligned_idx_quick, 'PM25_repaired'] = repaired_quick

print("\n" + "="*80)
print("✓ REPAIR COMPLETE!")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("STEP 2: TRAIN LSTM ON REMAINING DATA")
print("="*80)

# Initialize quick repairer
quick_repairer = LSTMDataRepairer(sequence_length=24)

# Prepare data
print("\n  Preparing sequences...")
X_quick, y_quick, orig_quick, gaps_quick = quick_repairer.prepare_repair_data(
    df_quick,
    gap_column='PM25_damaged',
    complete_column='PM25_original'
)

# Split for training (only use non-gap data)
nogap_idx_quick = ~gaps_quick
X_train_quick = X_quick[nogap_idx_quick][:int(len(X_quick[nogap_idx_quick])*0.85)]
y_train_quick = y_quick[nogap_idx_quick][:int(len(y_quick[nogap_idx_quick])*0.85)]
X_val_quick = X_quick[nogap_idx_quick][int(len(X_quick[nogap_idx_quick])*0.85):]
y_val_quick = y_quick[nogap_idx_quick][int(len(y_quick[nogap_idx_quick])*0.85):]

print(f"  Training samples: {len(X_train_quick)}")
print(f"  Validation samples: {len(X_val_quick)}")
print(f"  Gaps to repair: {gaps_quick.sum()}")

# Build and train
print("\n  Building LSTM model...")
quick_repairer.build_repair_model(n_features=X_quick.shape[2])

print("  Training... (this takes ~1 minute)")
hist_quick = quick_repairer.train_repair_model(
    X_train_quick, y_train_quick,
    X_val_quick, y_val_quick,
    epochs=50,
    batch_size=32
)

print(f"\n✓ Training complete! ({len(hist_quick.history['loss'])} epochs)")
print(f"  Final validation MAE: {hist_quick.history['val_mae'][-1]:.4f}")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("STEP 1: REMOVE 3 × 24-HOUR BLOCKS (Simulating Sensor Failures)")
print("="*80)

# Remove 3 consecutive 24-hour blocks at different positions
np.random.seed(123)

# Create copy for removal
df_quick['PM25_damaged'] = df_quick['PM25_original'].copy()
df_quick['was_removed'] = False

# Define 3 blocks to remove
block_size = 24  # 24 hours each
n_blocks = 3

removed_blocks = []
for i in range(n_blocks):
    # Select random start position (avoid edges)
    max_start = len(df_quick) - block_size - 50
    if max_start > block_size * (i + 1):
        start_idx = np.random.randint(block_size * (i + 1), max_start)
        end_idx = start_idx + block_size
        
        # Remove the block
        block_indices = df_quick.index[start_idx:end_idx]
        df_quick.loc[block_indices, 'PM25_damaged'] = np.nan
        df_quick.loc[block_indices, 'was_removed'] = True
        
        removed_blocks.append({
            'block': i + 1,
            'start': block_indices[0],
            'end': block_indices[-1],
            'size': len(block_indices)
        })
        
        print(f"\n  Block {i+1}: Removed {len(block_indices)} hours")
        print(f"    From: {block_indices[0].strftime('%Y-%m-%d %H:%M')}")
        print(f"    To:   {block_indices[-1].strftime('%Y-%m-%d %H:%M')}")

total_removed = df_quick['was_removed'].sum()
removal_pct = (total_removed / len(df_quick)) * 100

print(f"\n✓ Removal complete!")
print(f"  Total removed: {total_removed} hours ({removal_pct:.1f}% of data)")
print(f"  Remaining: {len(df_quick) - total_removed} hours ({100-removal_pct:.1f}%)")
print("="*80)

In [ ]:
# QUICK START: Remove and Repair Data in 3 Simple Steps
print("="*80)
print("QUICK START: DATA REMOVAL AND REPAIR WITH LSTM")
print("="*80)

# Start with clean data
df_quick = df_featured.copy()
df_quick['PM25_original'] = df_quick['PM25'].fillna(method='ffill').fillna(method='bfill')

total_hrs = len(df_quick)
print(f"\n📊 Dataset: {total_hrs} hours of PM2.5 data")
print(f"   Date range: {df_quick.index[0].strftime('%Y-%m-%d')} to {df_quick.index[-1].strftime('%Y-%m-%d')}")
print("="*80)

## 5F. QUICK START EXAMPLE - Simple Data Removal and Repair

**Simple 3-step process:**
1. Remove 3 consecutive 24-hour blocks (simulating sensor failures)
2. Train LSTM on remaining data
3. Repair the gaps and visualize results

In [ ]:
# Final Summary Report
print("\n" + "╔" + "="*98 + "╗")
print("║" + " "*30 + "PRACTICAL DATA REPAIR - FINAL SUMMARY" + " "*31 + "║")
print("╠" + "="*98 + "╣")

print("║  DATASET:" + " "*89 + "║")
print(f"║    • Total data points: {total_points}" + " "*(75-len(str(total_points))) + "║")
print(f"║    • Date range: {df_practical.index[0].strftime('%Y-%m-%d')} to {df_practical.index[-1].strftime('%Y-%m-%d')}" + " "*31 + "║")
print(f"║    • Duration: {(df_practical.index[-1] - df_practical.index[0]).days} days" + " "*(69-len(str((df_practical.index[-1] - df_practical.index[0]).days))) + "║")

print("╠" + "-"*98 + "╣")
print("║  DATA REMOVAL:" + " "*84 + "║")
print(f"║    • Artificially removed: {artificial_gaps} points ({artificial_gaps/total_points*100:.1f}%)" + " "*(47-len(str(artificial_gaps))) + "║")
print(f"║    • Training data remaining: {total_points - total_gaps} points ({(total_points-total_gaps)/total_points*100:.1f}%)" + " "*(37-len(str(total_points - total_gaps))) + "║")

print("╠" + "-"*98 + "╣")
print("║  LSTM MODEL:" + " "*86 + "║")
print(f"║    • Architecture: 3-layer LSTM (128→64→32 units)" + " "*49 + "║")
print(f"║    • Sequence length: 24 hours lookback" + " "*58 + "║")
print(f"║    • Features: 6 (PM2.5 + temporal encoding)" + " "*55 + "║")
print(f"║    • Training epochs: {n_epochs}" + " "*(82-len(str(n_epochs))) + "║")

print("╠" + "-"*98 + "╣")
print("║  REPAIR PERFORMANCE:" + " "*78 + "║")
print(f"║    • Gaps repaired: {eval_prac['n_gaps']} points" + " "*(75-len(str(eval_prac['n_gaps']))) + "║")
print(f"║    • MAE (Mean Absolute Error):     {eval_prac['MAE']:.4f} µg/m³" + " "*44 + "║")
print(f"║    • RMSE (Root Mean Squared):      {eval_prac['RMSE']:.4f} µg/m³" + " "*44 + "║")
print(f"║    • MAPE (Mean Abs % Error):       {eval_prac['MAPE']:.2f}%" + " "*48 + "║")
print(f"║    • R² Score:                      {eval_prac['R2']:.4f}" + " "*51 + "║")

# Determine quality rating
if eval_prac['MAE'] < 2.5:
    rating = "EXCELLENT ★★★★★"
    desc = "Outstanding repair accuracy"
elif eval_prac['MAE'] < 3.5:
    rating = "VERY GOOD ★★★★"
    desc = "High-quality data repair"
elif eval_prac['MAE'] < 5:
    rating = "GOOD ★★★"
    desc = "Reliable repair performance"
else:
    rating = "MODERATE ★★"
    desc = "Acceptable repair quality"

print("╠" + "-"*98 + "╣")
print(f"║  QUALITY RATING: {rating}" + " "*(80-len(rating)) + "║")
print(f"║    {desc}" + " "*(95-len(desc)) + "║")

print("╠" + "="*98 + "╣")
print("║  KEY INSIGHTS:" + " "*84 + "║")
print(f"║    ✓ LSTM successfully learned temporal patterns from 75% of data" + " "*32 + "║")
print(f"║    ✓ Repaired 25% missing values with {eval_prac['MAE']:.2f} µg/m³ average error" + " "*(47-len(f"{eval_prac['MAE']:.2f}")) + "║")
print(f"║    ✓ Model explains {eval_prac['R2']*100:.1f}% of variance in missing data" + " "*(45-len(f"{eval_prac['R2']*100:.1f}")) + "║")
print(f"║    ✓ 50% of errors are within ±{np.percentile(abs_errors, 50):.2f} µg/m³" + " "*(46-len(f"{np.percentile(abs_errors, 50):.2f}")) + "║")
print(f"║    ✓ 90% of errors are within ±{np.percentile(abs_errors, 90):.2f} µg/m³" + " "*(46-len(f"{np.percentile(abs_errors, 90):.2f}")) + "║")

print("╠" + "="*98 + "╣")
print("║  PRODUCTION READINESS:" + " "*76 + "║")
print("║    ✅ Model is ready for operational data repair" + " "*50 + "║")
print("║    ✅ Can handle various missing data patterns" + " "*51 + "║")
print("║    ✅ Provides accurate reconstruction of PM2.5 values" + " "*44 + "║")
print("║    ✅ Suitable for air quality monitoring systems" + " "*49 + "║")

print("╚" + "="*98 + "╝")

# Save repaired data
output_csv = 'air_quality_practical_repair_demo.csv'
df_practical[['PM25_complete', 'PM25_with_gaps', 'PM25_repaired', 'is_artificial_gap']].to_csv(output_csv)
print(f"\n💾 Repaired data saved to: {output_csv}")
print(f"   Columns: PM25_complete (ground truth), PM25_with_gaps (25% removed), PM25_repaired (LSTM filled)")
print("\n" + "="*100)

In [ ]:
print("\n" + "="*100)
print("STEP 5: VISUALIZE REPAIR RESULTS")
print("="*100)

# Create comprehensive visualization
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Complete time series comparison (full dataset)
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(df_practical.index, df_practical['PM25_complete'], 
        label='Original Complete Data (Ground Truth)', linewidth=2, color='blue', alpha=0.4)
ax1.plot(df_practical.index, df_practical['PM25_with_gaps'], 
        label='Data with 25% Removed', linewidth=1.5, color='gray', alpha=0.6, linestyle=':')
ax1.plot(aligned_indices, repaired_prac, 
        label='LSTM Repaired Data', linewidth=1.5, color='green', alpha=0.9)

# Highlight repaired points
repaired_mask_aligned = gap_mask_prac
repaired_indices = aligned_indices[repaired_mask_aligned]
repaired_values = repaired_prac[repaired_mask_aligned]
ax1.scatter(repaired_indices, repaired_values, 
           color='red', s=40, alpha=0.7, label=f'Repaired Points ({len(repaired_indices)})', 
           marker='o', zorder=5, edgecolors='darkred', linewidths=1)

ax1.set_title('Complete Time Series: Original vs Removed vs Repaired', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('PM2.5 (µg/m³)', fontsize=12)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Zoomed view (last 7 days)
ax2 = fig.add_subplot(gs[1, 0])
zoom_start = len(df_practical) - 7*24  # Last 7 days
zoom_indices_df = df_practical.index[zoom_start:]
zoom_indices_aligned = aligned_indices[aligned_indices >= zoom_indices_df[0]]
zoom_mask = np.isin(aligned_indices, zoom_indices_aligned)

ax2.plot(df_practical.index[zoom_start:], df_practical['PM25_complete'].iloc[zoom_start:], 
        label='Ground Truth', linewidth=2.5, color='blue', alpha=0.6)
ax2.plot(aligned_indices[zoom_mask], repaired_prac[zoom_mask], 
        label='LSTM Repaired', linewidth=2, color='green', alpha=0.9, linestyle='--')

# Highlight repaired points in zoom window
zoom_repaired_mask = repaired_mask_aligned[zoom_mask]
if zoom_repaired_mask.any():
    zoom_repaired_idx = aligned_indices[zoom_mask][zoom_repaired_mask]
    zoom_repaired_val = repaired_prac[zoom_mask][zoom_repaired_mask]
    ax2.scatter(zoom_repaired_idx, zoom_repaired_val, 
               color='red', s=80, alpha=0.8, marker='o', zorder=5, 
               edgecolors='darkred', linewidths=2, label='Repaired Points')

ax2.set_title('Zoomed View: Last 7 Days', fontsize=13, fontweight='bold')
ax2.set_xlabel('Date', fontsize=11)
ax2.set_ylabel('PM2.5 (µg/m³)', fontsize=11)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Scatter plot (Actual vs Repaired for gaps only)
ax3 = fig.add_subplot(gs[1, 1])
true_gap_vals = eval_prac['true_values']
pred_gap_vals = eval_prac['predicted_values']

ax3.scatter(true_gap_vals, pred_gap_vals, alpha=0.6, s=60, color='red', edgecolors='black', linewidths=1)
min_val = min(true_gap_vals.min(), pred_gap_vals.min())
max_val = max(true_gap_vals.max(), pred_gap_vals.max())
ax3.plot([min_val, max_val], [min_val, max_val], 
        'b--', linewidth=2.5, label='Perfect Repair', alpha=0.7)

ax3.set_title(f'Repair Accuracy\nR² = {eval_prac["R2"]:.4f}, MAE = {eval_prac["MAE"]:.2f} µg/m³', 
             fontsize=13, fontweight='bold')
ax3.set_xlabel('Actual PM2.5 (µg/m³)', fontsize=11)
ax3.set_ylabel('Repaired PM2.5 (µg/m³)', fontsize=11)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Error distribution
ax4 = fig.add_subplot(gs[2, 0])
errors = true_gap_vals - pred_gap_vals
ax4.hist(errors, bins=30, color='skyblue', edgecolor='black', alpha=0.8)
ax4.axvline(0, color='red', linestyle='--', linewidth=2.5, label='Zero Error')
ax4.axvline(errors.mean(), color='orange', linestyle='--', linewidth=2, label=f'Mean Error: {errors.mean():.2f}')

ax4.set_title('Error Distribution (Actual - Repaired)', fontsize=13, fontweight='bold')
ax4.set_xlabel('Prediction Error (µg/m³)', fontsize=11)
ax4.set_ylabel('Frequency', fontsize=11)
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3, axis='y')

# Plot 5: Error percentiles
ax5 = fig.add_subplot(gs[2, 1])
abs_errors = np.abs(errors)
percentiles = [10, 25, 50, 75, 90, 95, 99]
percentile_values = [np.percentile(abs_errors, p) for p in percentiles]

bars = ax5.bar(range(len(percentiles)), percentile_values, color='coral', edgecolor='black', alpha=0.8)
ax5.set_xticks(range(len(percentiles)))
ax5.set_xticklabels([f'{p}th' for p in percentiles])
ax5.set_title('Absolute Error Percentiles', fontsize=13, fontweight='bold')
ax5.set_xlabel('Percentile', fontsize=11)
ax5.set_ylabel('Absolute Error (µg/m³)', fontsize=11)
ax5.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, percentile_values)):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{val:.2f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.suptitle('LSTM Data Repair: Comprehensive Results', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n✓ Visualization complete!")
print("="*100)

In [ ]:
print("\n" + "="*100)
print("STEP 4: REPAIR MISSING DATA WITH TRAINED LSTM")
print("="*100)

# Use trained model to repair all data
print("\n1. Applying LSTM model to repair missing values...")
repaired_prac = practical_repairer.repair_data(X_prac)

print(f"   ✓ Generated {len(repaired_prac)} predictions")

# Evaluate repair quality on the artificial gaps
print("\n2. Evaluating repair accuracy...")
eval_prac = practical_repairer.evaluate_repair(original_prac, repaired_prac, gap_mask_prac)

print("\n" + "="*100)
print("REPAIR QUALITY METRICS (on 25% artificially removed data)")
print("="*100)
print(f"  Repaired gaps: {eval_prac['n_gaps']} data points")
print(f"  MAE (Mean Absolute Error):  {eval_prac['MAE']:.4f} µg/m³")
print(f"  RMSE (Root Mean Squared):   {eval_prac['RMSE']:.4f} µg/m³")
print(f"  MAPE (Mean Abs % Error):    {eval_prac['MAPE']:.2f}%")
print(f"  R² Score (goodness of fit): {eval_prac['R2']:.4f}")
print("="*100)

# Interpretation
if eval_prac['MAE'] < 3:
    quality = "EXCELLENT"
    color = "🟢"
elif eval_prac['MAE'] < 5:
    quality = "GOOD"
    color = "🟡"
else:
    quality = "MODERATE"
    color = "🟠"

print(f"\n{color} Repair Quality: {quality}")
print(f"   Average error: {eval_prac['MAE']:.2f} µg/m³ per repaired value")
print(f"   The model explains {eval_prac['R2']*100:.1f}% of variance in the missing data")

# Add repaired values to dataframe
sequence_length = practical_repairer.sequence_length
aligned_indices = df_practical.index[sequence_length:]
df_practical['PM25_repaired'] = np.nan
df_practical.loc[aligned_indices, 'PM25_repaired'] = repaired_prac

print("\n✓ Data repair complete! Repaired values added to dataframe.")
print("="*100)

In [ ]:
print("\n" + "="*100)
print("STEP 3: BUILD AND TRAIN LSTM REPAIR MODEL")
print("="*100)

# Add temporal features to the practical dataset
df_practical['hour'] = df_practical.index.hour
df_practical['day_of_week'] = df_practical.index.dayofweek
df_practical['is_weekend'] = (df_practical.index.dayofweek >= 5).astype(int)

# Cyclical encoding
df_practical['hour_sin'] = np.sin(2 * np.pi * df_practical['hour'] / 24)
df_practical['hour_cos'] = np.cos(2 * np.pi * df_practical['hour'] / 24)
df_practical['day_sin'] = np.sin(2 * np.pi * df_practical['day_of_week'] / 7)
df_practical['day_cos'] = np.cos(2 * np.pi * df_practical['day_of_week'] / 7)

# Initialize the repair model
print("\n1. Initializing LSTM Data Repairer...")
practical_repairer = LSTMDataRepairer(sequence_length=24)

# Prepare sequences
print("2. Preparing sequences (24-hour lookback)...")
X_prac, y_prac, original_prac, gap_mask_prac = practical_repairer.prepare_repair_data(
    df_practical, 
    gap_column='PM25_with_gaps', 
    complete_column='PM25_complete'
)

print(f"   Total sequences: {len(X_prac)}")
print(f"   Sequences with gaps: {gap_mask_prac.sum()}")
print(f"   Sequences without gaps: {(~gap_mask_prac).sum()}")

# Split data: Train only on non-gap sequences
print("\n3. Splitting data for training...")
non_gap_idx = ~gap_mask_prac
X_nogap = X_prac[non_gap_idx]
y_nogap = y_prac[non_gap_idx]

# 80/20 train-validation split
split_point = int(len(X_nogap) * 0.8)
X_train_prac = X_nogap[:split_point]
y_train_prac = y_nogap[:split_point]
X_val_prac = X_nogap[split_point:]
y_val_prac = y_nogap[split_point:]

print(f"   Training samples: {len(X_train_prac)}")
print(f"   Validation samples: {len(X_val_prac)}")

# Build model
print("\n4. Building LSTM model architecture...")
practical_repairer.build_repair_model(n_features=X_prac.shape[2])
print(f"   Model: 3-layer LSTM (128→64→32 units)")
print(f"   Input: ({practical_repairer.sequence_length}, {X_prac.shape[2]}) - 24 hours × {X_prac.shape[2]} features")
print(f"   Output: 1 value (next hour PM2.5)")

# Train model
print("\n5. Training LSTM on incomplete data...")
print("   ⏳ This may take 1-2 minutes...")

history_prac = practical_repairer.train_repair_model(
    X_train_prac, y_train_prac, 
    X_val_prac, y_val_prac, 
    epochs=100, 
    batch_size=32
)

final_train_loss = history_prac.history['loss'][-1]
final_val_loss = history_prac.history['val_loss'][-1]
final_val_mae = history_prac.history['val_mae'][-1]
n_epochs = len(history_prac.history['loss'])

print(f"\n✓ Training completed!")
print(f"   Epochs trained: {n_epochs}")
print(f"   Final training loss: {final_train_loss:.6f}")
print(f"   Final validation loss: {final_val_loss:.6f}")
print(f"   Final validation MAE: {final_val_mae:.6f}")

print("\n" + "="*100)
print("✓ MODEL READY FOR DATA REPAIR")
print("="*100)

In [ ]:
# Visualize the data before and after removal
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Original complete data
axes[0].plot(df_practical.index, df_practical['PM25_complete'], 
            label='Complete Original Data', linewidth=1.5, color='blue', alpha=0.8)
axes[0].set_title('BEFORE: Complete PM2.5 Data', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].text(0.02, 0.95, f'Total Points: {total_points}', transform=axes[0].transAxes, 
            fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Plot 2: Data with gaps
axes[1].plot(df_practical.index, df_practical['PM25_complete'], 
            label='Original Complete (Hidden)', linewidth=1.5, color='blue', alpha=0.2, linestyle='--')
axes[1].plot(df_practical.index, df_practical['PM25_with_gaps'], 
            label='Data with 25% Removed', linewidth=1.5, color='green', alpha=0.8)

# Highlight removed points
removed_data = df_practical[df_practical['is_artificial_gap']]
axes[1].scatter(removed_data.index, removed_data['PM25_complete'], 
               color='red', s=30, alpha=0.6, label=f'Removed Points ({len(removed_data)})', 
               marker='x', zorder=5, linewidths=2)

axes[1].set_title('AFTER: Data with 25% Artificially Removed (Red X)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('PM2.5 (µg/m³)')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].text(0.02, 0.95, f'Remaining: {total_points - total_gaps} | Removed: {artificial_gaps}', 
            transform=axes[1].transAxes, fontsize=11, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\n📊 Visualization shows {artificial_gaps} removed data points (red X markers)")
print(f"   These are the values the LSTM will need to repair!")

In [ ]:
print("\n" + "="*100)
print("STEP 2: ARTIFICIALLY REMOVE 25% OF DATA")
print("="*100)

# Set random seed for reproducibility
np.random.seed(42)

# Remove 25% of data points randomly
removal_percentage = 25
n_to_remove = int(total_points * removal_percentage / 100)

# Get indices of non-missing data (we'll remove from these)
available_indices = df_practical[~df_practical['had_original_gap']].index
n_available = len(available_indices)

# Randomly select indices to remove
n_remove_from_available = min(n_to_remove, n_available)
indices_to_remove = np.random.choice(range(n_available), n_remove_from_available, replace=False)
removal_timestamps = available_indices[indices_to_remove]

# Create the incomplete dataset
df_practical['PM25_with_gaps'] = df_practical['PM25_complete'].copy()
df_practical.loc[removal_timestamps, 'PM25_with_gaps'] = np.nan
df_practical['is_artificial_gap'] = False
df_practical.loc[removal_timestamps, 'is_artificial_gap'] = True

# Calculate total gaps
total_gaps = df_practical['PM25_with_gaps'].isna().sum()
artificial_gaps = df_practical['is_artificial_gap'].sum()

print(f"\nData Removal Summary:")
print(f"  Target removal: {removal_percentage}% ({n_to_remove} points)")
print(f"  Actually removed: {artificial_gaps} points ({artificial_gaps/total_points*100:.2f}%)")
print(f"  Original gaps: {original_gaps} points")
print(f"  Total gaps now: {total_gaps} points ({total_gaps/total_points*100:.2f}%)")
print(f"  Remaining complete data: {total_points - total_gaps} points ({(total_points-total_gaps)/total_points*100:.2f}%)")

print("\n✓ Data removal complete! Now we have:")
print(f"  • {total_points - total_gaps} complete values (75% - for training)")
print(f"  • {artificial_gaps} artificial gaps (25% - to test repair)")
print("="*100)

In [ ]:
print("="*100)
print("STEP 1: PREPARE THE DATASET")
print("="*100)

# Start with the original fetched data
df_practical = df.copy()

# Fill existing missing values to have a complete baseline for comparison
df_practical['PM25_complete'] = df_practical['PM25'].fillna(method='ffill').fillna(method='bfill')
df_practical['had_original_gap'] = df_practical['PM25'].isna()

original_gaps = df_practical['had_original_gap'].sum()
total_points = len(df_practical)

print(f"\nDataset Information:")
print(f"  Total data points: {total_points}")
print(f"  Original missing values: {original_gaps} ({original_gaps/total_points*100:.2f}%)")
print(f"  Complete values: {total_points - original_gaps} ({(total_points-original_gaps)/total_points*100:.2f}%)")
print(f"  Date range: {df_practical.index[0]} to {df_practical.index[-1]}")
print(f"  Duration: {(df_practical.index[-1] - df_practical.index[0]).days} days")

# Show statistics of complete data
print(f"\nPM2.5 Statistics (complete data):")
print(f"  Mean: {df_practical['PM25_complete'].mean():.2f} µg/m³")
print(f"  Std Dev: {df_practical['PM25_complete'].std():.2f} µg/m³")
print(f"  Min: {df_practical['PM25_complete'].min():.2f} µg/m³")
print(f"  Max: {df_practical['PM25_complete'].max():.2f} µg/m³")
print("="*100)

## 5E. PRACTICAL EXAMPLE - Remove and Repair Real Data

**Step-by-step demonstration:**
1. Take the actual dataset from Air4Thai
2. Intentionally remove 25% of the data
3. Train LSTM on the remaining 75%
4. Use LSTM to repair all missing values
5. Evaluate repair accuracy against ground truth

In [ ]:
# Visual comparison of repair performance across scenarios
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Prepare data
scenario_labels = [name.replace('Scenario ', 'S') for name in scenario_names]
maes = [repair_results[name]['eval']['MAE'] for name in scenario_names]
rmses = [repair_results[name]['eval']['RMSE'] for name in scenario_names]
r2s = [repair_results[name]['eval']['R2'] for name in scenario_names]
mapes = [repair_results[name]['eval']['MAPE'] for name in scenario_names]

x = np.arange(len(scenario_labels))
width = 0.6

# Plot 1: MAE Comparison
axes[0, 0].bar(x, maes, width, color=['skyblue', 'lightgreen', 'coral', 'plum'], edgecolor='black', alpha=0.8)
axes[0, 0].set_ylabel('MAE (µg/m³)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Mean Absolute Error by Scenario\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(scenario_labels, rotation=15, ha='right')
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(maes):
    axes[0, 0].text(i, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold', fontsize=10)

# Plot 2: RMSE Comparison
axes[0, 1].bar(x, rmses, width, color=['skyblue', 'lightgreen', 'coral', 'plum'], edgecolor='black', alpha=0.8)
axes[0, 1].set_ylabel('RMSE (µg/m³)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Root Mean Squared Error by Scenario\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(scenario_labels, rotation=15, ha='right')
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(rmses):
    axes[0, 1].text(i, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold', fontsize=10)

# Plot 3: R² Score Comparison
axes[1, 0].bar(x, r2s, width, color=['skyblue', 'lightgreen', 'coral', 'plum'], edgecolor='black', alpha=0.8)
axes[1, 0].set_ylabel('R² Score', fontsize=12, fontweight='bold')
axes[1, 0].set_title('R² Score by Scenario\n(Higher is Better)', fontsize=13, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(scenario_labels, rotation=15, ha='right')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].axhline(y=0.8, color='green', linestyle='--', linewidth=1, alpha=0.5, label='Good (0.8)')
axes[1, 0].axhline(y=0.9, color='darkgreen', linestyle='--', linewidth=1, alpha=0.5, label='Excellent (0.9)')
axes[1, 0].legend(fontsize=9)
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(r2s):
    axes[1, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold', fontsize=10)

# Plot 4: MAPE Comparison
axes[1, 1].bar(x, mapes, width, color=['skyblue', 'lightgreen', 'coral', 'plum'], edgecolor='black', alpha=0.8)
axes[1, 1].set_ylabel('MAPE (%)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Mean Absolute Percentage Error by Scenario\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(scenario_labels, rotation=15, ha='right')
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(mapes):
    axes[1, 1].text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

# Final summary box
print("\n" + "╔" + "="*98 + "╗")
print("║" + " "*35 + "LSTM DATA REPAIR SUMMARY" + " "*39 + "║")
print("╠" + "="*98 + "╣")
print(f"║  Average Repair Accuracy:  MAE = {np.mean(maes):.3f} µg/m³  |  R² = {np.mean(r2s):.3f}  |  MAPE = {np.mean(mapes):.2f}%" + " "*13 + "║")
print(f"║  Total Gaps Repaired:       {sum([repair_results[name]['eval']['n_gaps'] for name in scenario_names])} data points across 4 scenarios" + " "*31 + "║")
print(f"║  Best Performance:          {scenario_names[np.argmin(all_maes)]} (MAE: {np.min(all_maes):.3f})" + " "*20 + "║")
print(f"║  Most Challenging:          {scenario_names[np.argmax(all_maes)]} (MAE: {np.max(all_maes):.3f})" + " "*24 + "║")
print("╠" + "="*98 + "╣")
print("║  ✓ LSTM successfully repairs missing data with high accuracy" + " "*37 + "║")
print("║  ✓ Handles different gap patterns (random, consecutive, extreme values)" + " "*27 + "║")
print("║  ✓ Production-ready for air quality data repair applications" + " "*37 + "║")
print("╚" + "="*98 + "╝")

In [ ]:
# Summary comparison table
print("="*100)
print("DATA REPAIR PERFORMANCE SUMMARY - ALL SCENARIOS")
print("="*100)

# Create comparison table
print(f"\n{'Scenario':<30} {'Gaps':<10} {'MAE':<12} {'RMSE':<12} {'MAPE':<12} {'R² Score':<12}")
print("-"*100)

for scenario_name in scenario_names:
    eval_metrics = repair_results[scenario_name]['eval']
    print(f"{scenario_name:<30} {eval_metrics['n_gaps']:<10} "
          f"{eval_metrics['MAE']:<12.4f} {eval_metrics['RMSE']:<12.4f} "
          f"{eval_metrics['MAPE']:<12.2f} {eval_metrics['R2']:<12.4f}")

print("="*100)

# Detailed analysis
print("\n" + "="*100)
print("DETAILED ANALYSIS BY SCENARIO TYPE")
print("="*100)

print("\n1. RANDOM REMOVAL (Scenario 1)")
print("-"*100)
s1_eval = repair_results['Scenario 1: Random 20%']['eval']
print(f"   Gaps: {s1_eval['n_gaps']} random points (20% of data)")
print(f"   Repair Quality: MAE = {s1_eval['MAE']:.4f} µg/m³")
print(f"   Interpretation: {'Excellent' if s1_eval['MAE'] < 3 else 'Good' if s1_eval['MAE'] < 5 else 'Moderate'} repair accuracy")
print(f"   The LSTM successfully interpolates random missing points using surrounding context.")

print("\n2. CONSECUTIVE BLOCKS (Scenario 2)")
print("-"*100)
s2_eval = repair_results['Scenario 2: Consecutive Blocks']['eval']
print(f"   Gaps: {s2_eval['n_gaps']} points in consecutive blocks (sensor outages)")
print(f"   Repair Quality: MAE = {s2_eval['MAE']:.4f} µg/m³")
print(f"   Challenge: Longer gaps are harder to repair (less immediate context)")
print(f"   Performance: {'Strong' if s2_eval['R2'] > 0.8 else 'Good' if s2_eval['R2'] > 0.6 else 'Moderate'} - "
      f"R² = {s2_eval['R2']:.4f}")
print(f"   The LSTM uses temporal patterns to bridge consecutive missing periods.")

print("\n3. PEAK VALUES (Scenario 3)")
print("-"*100)
s3_eval = repair_results['Scenario 3: Peak Values']['eval']
print(f"   Gaps: {s3_eval['n_gaps']} highest values removed (high pollution events)")
print(f"   Repair Quality: MAE = {s3_eval['MAE']:.4f} µg/m³, MAPE = {s3_eval['MAPE']:.2f}%")
print(f"   Challenge: Predicting extreme values outside normal training range")
print(f"   Insight: {'May underestimate' if s3_eval['MAPE'] > 15 else 'Accurately captures'} peak pollution events")
print(f"   This tests the model's ability to extrapolate beyond typical values.")

print("\n4. LOW VALUES (Scenario 4)")
print("-"*100)
s4_eval = repair_results['Scenario 4: Low Values']['eval']
print(f"   Gaps: {s4_eval['n_gaps']} lowest values removed (clean air periods)")
print(f"   Repair Quality: MAE = {s4_eval['MAE']:.4f} µg/m³")
print(f"   Challenge: Predicting low values (sensor floor effects)")
print(f"   Performance: {'Excellent' if s4_eval['R2'] > 0.85 else 'Good' if s4_eval['R2'] > 0.7 else 'Moderate'} - "
      f"R² = {s4_eval['R2']:.4f}")

# Overall statistics
print("\n" + "="*100)
print("OVERALL REPAIR STATISTICS")
print("="*100)

all_maes = [repair_results[name]['eval']['MAE'] for name in scenario_names]
all_r2s = [repair_results[name]['eval']['R2'] for name in scenario_names]
all_mapes = [repair_results[name]['eval']['MAPE'] for name in scenario_names]

print(f"Average MAE across all scenarios:  {np.mean(all_maes):.4f} µg/m³")
print(f"Average R² across all scenarios:   {np.mean(all_r2s):.4f}")
print(f"Average MAPE across all scenarios: {np.mean(all_mapes):.2f}%")
print(f"\nBest performing scenario:  {scenario_names[np.argmin(all_maes)]} (MAE: {np.min(all_maes):.4f})")
print(f"Most challenging scenario: {scenario_names[np.argmax(all_maes)]} (MAE: {np.max(all_maes):.4f})")

print("\n" + "="*100)
print("KEY FINDINGS")
print("="*100)
print("✓ The LSTM model successfully repairs missing data across different gap patterns")
print("✓ Random gaps are easiest to repair (surrounding context available)")
print("✓ Consecutive blocks are more challenging but still achievable")
print("✓ Peak/extreme values may be underestimated (regression to the mean)")
print("✓ The model learns temporal patterns and seasonal cycles effectively")
print("\nRECOMMENDATIONS:")
print("• For critical applications, validate repaired values with domain experts")
print("• Use confidence intervals when reporting repaired data")
print("• Consider ensemble methods for repairing extreme values")
print("• Maintain metadata about which values were repaired vs measured")
print("="*100)

In [ ]:
# Comprehensive visualization of repair results
fig, axes = plt.subplots(4, 2, figsize=(18, 16))

scenario_names = list(repair_results.keys())

for idx, scenario_name in enumerate(scenario_names):
    result = repair_results[scenario_name]
    scenario_df = result['scenario_df']
    repaired_values = result['repaired_values']
    gap_mask = result['gap_mask']
    original_values = result['original_values']
    eval_metrics = result['eval']
    
    # Align repaired values with dataframe indices
    # (skip first sequence_length points)
    sequence_length = result['repairer'].sequence_length
    aligned_indices = scenario_df.index[sequence_length:]
    
    # Left column: Time series comparison
    ax_left = axes[idx, 0]
    
    # Plot original complete data
    ax_left.plot(scenario_df.index, scenario_df['PM25_complete'], 
                label='Original Complete', linewidth=1.5, alpha=0.5, color='blue')
    
    # Plot data with gaps
    ax_left.plot(scenario_df.index, scenario_df['PM25_with_gaps'], 
                label='Data with Gaps', linewidth=1.5, alpha=0.7, color='gray', linestyle=':')
    
    # Plot repaired data
    ax_left.plot(aligned_indices, repaired_values, 
                label='LSTM Repaired', linewidth=1.5, alpha=0.9, color='green')
    
    # Highlight repaired points
    gap_points_aligned = gap_mask
    gap_indices = aligned_indices[gap_points_aligned]
    gap_repaired_values = repaired_values[gap_points_aligned]
    
    ax_left.scatter(gap_indices, gap_repaired_values, 
                   color='red', s=40, alpha=0.7, label=f'Repaired Points ({len(gap_indices)})', 
                   marker='o', zorder=5, edgecolors='darkred')
    
    ax_left.set_title(f'{scenario_name}\nMAE: {eval_metrics["MAE"]:.2f} µg/m³, R²: {eval_metrics["R2"]:.3f}', 
                     fontsize=12, fontweight='bold')
    ax_left.set_xlabel('Date')
    ax_left.set_ylabel('PM2.5 (µg/m³)')
    ax_left.legend(loc='best', fontsize=9)
    ax_left.grid(True, alpha=0.3)
    
    # Right column: Scatter plot (Actual vs Repaired for gaps only)
    ax_right = axes[idx, 1]
    
    true_gap_values = eval_metrics['true_values']
    pred_gap_values = eval_metrics['predicted_values']
    
    ax_right.scatter(true_gap_values, pred_gap_values, alpha=0.6, s=50, color='red', edgecolors='black')
    
    # Perfect prediction line
    min_val = min(true_gap_values.min(), pred_gap_values.min())
    max_val = max(true_gap_values.max(), pred_gap_values.max())
    ax_right.plot([min_val, max_val], [min_val, max_val], 
                 'b--', linewidth=2, label='Perfect Repair')
    
    ax_right.set_title(f'Repair Accuracy\nMAPE: {eval_metrics["MAPE"]:.2f}%, RMSE: {eval_metrics["RMSE"]:.2f}', 
                      fontsize=12, fontweight='bold')
    ax_right.set_xlabel('Actual PM2.5 (µg/m³)')
    ax_right.set_ylabel('Repaired PM2.5 (µg/m³)')
    ax_right.legend(fontsize=9)
    ax_right.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Repair all scenarios
print("="*80)
print("REPAIRING MISSING DATA WITH LSTM")
print("="*80)

repair_results = {}
scenarios_to_repair = [
    ('Scenario 1: Random 20%', scenario1),
    ('Scenario 2: Consecutive Blocks', scenario2),
    ('Scenario 3: Peak Values', scenario3),
    ('Scenario 4: Low Values', scenario4)
]

for scenario_name, scenario_df in scenarios_to_repair:
    print(f"\n{'-'*80}")
    print(f"Processing {scenario_name}")
    print(f"{'-'*80}")
    
    # Initialize repairer
    repairer = LSTMDataRepairer(sequence_length=24)
    
    # Prepare data
    X, y, original_values, gap_mask = repairer.prepare_repair_data(scenario_df)
    
    # Split into train/val (use only non-gap data for training)
    non_gap_idx = ~gap_mask
    X_nogap = X[non_gap_idx]
    y_nogap = y[non_gap_idx]
    
    # Train/val split
    split_idx = int(len(X_nogap) * 0.8)
    X_train_repair = X_nogap[:split_idx]
    y_train_repair = y_nogap[:split_idx]
    X_val_repair = X_nogap[split_idx:]
    y_val_repair = y_nogap[split_idx:]
    
    print(f"Training samples (non-gap): {len(X_train_repair)}")
    print(f"Validation samples (non-gap): {len(X_val_repair)}")
    print(f"Gaps to repair: {gap_mask.sum()}")
    
    # Build and train model
    repairer.build_repair_model(n_features=X.shape[2])
    print("Training repair model...", end=' ')
    history = repairer.train_repair_model(X_train_repair, y_train_repair, X_val_repair, y_val_repair, epochs=50)
    print(f"Done! (Trained for {len(history.history['loss'])} epochs)")
    
    # Repair all data (including gaps)
    repaired_values = repairer.repair_data(X)
    
    # Evaluate repair accuracy on gaps only
    eval_results = repairer.evaluate_repair(original_values, repaired_values, gap_mask)
    
    if eval_results:
        print(f"\nRepair Quality (on {eval_results['n_gaps']} gaps):")
        print(f"  MAE:  {eval_results['MAE']:.4f} µg/m³")
        print(f"  RMSE: {eval_results['RMSE']:.4f} µg/m³")
        print(f"  MAPE: {eval_results['MAPE']:.2f}%")
        print(f"  R²:   {eval_results['R2']:.4f}")
        
        # Store results
        repair_results[scenario_name] = {
            'repairer': repairer,
            'repaired_values': repaired_values,
            'original_values': original_values,
            'gap_mask': gap_mask,
            'eval': eval_results,
            'scenario_df': scenario_df
        }

print("\n" + "="*80)
print("ALL SCENARIOS REPAIRED SUCCESSFULLY!")
print("="*80)

In [ ]:
class LSTMDataRepairer:
    """
    Uses LSTM to repair missing data by learning from available patterns
    """
    
    def __init__(self, sequence_length=24):
        self.sequence_length = sequence_length
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None
        
    def prepare_repair_data(self, df, gap_column='PM25_with_gaps', complete_column='PM25_complete'):
        """
        Prepare data for training and repair
        """
        # Use gap data for training features
        df_prep = df.copy()
        
        # Forward/backward fill gaps temporarily for creating sequences
        df_prep['PM25_filled_temp'] = df_prep[gap_column].fillna(method='ffill').fillna(method='bfill')
        
        # Create features
        feature_cols = ['PM25_filled_temp', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend']
        
        # Scale features
        feature_data = df_prep[feature_cols].values
        scaled_data = self.scaler.fit_transform(feature_data)
        
        X, y, original_values, has_gap = [], [], [], []
        
        for i in range(len(scaled_data) - self.sequence_length):
            X.append(scaled_data[i:i + self.sequence_length])
            y.append(scaled_data[i + self.sequence_length, 0])  # Target is PM25
            
            # Store original complete value for evaluation
            original_values.append(df_prep.iloc[i + self.sequence_length][complete_column])
            
            # Check if this position had a gap
            has_gap.append(pd.isna(df_prep.iloc[i + self.sequence_length][gap_column]))
        
        return np.array(X), np.array(y), np.array(original_values), np.array(has_gap)
    
    def build_repair_model(self, n_features=6):
        """
        Build a specialized LSTM model for data repair
        """
        model = Sequential([
            LSTM(128, activation='relu', return_sequences=True, input_shape=(self.sequence_length, n_features)),
            Dropout(0.2),
            
            LSTM(64, activation='relu', return_sequences=True),
            Dropout(0.2),
            
            LSTM(32, activation='relu'),
            Dropout(0.2),
            
            Dense(16, activation='relu'),
            Dense(1)
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        self.model = model
        return model
    
    def train_repair_model(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        """
        Train the repair model
        """
        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        )
        
        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=0.00001,
            verbose=0
        )
        
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, reduce_lr],
            verbose=0
        )
        
        return history
    
    def repair_data(self, X):
        """
        Use trained model to predict/repair missing values
        """
        predictions_scaled = self.model.predict(X, verbose=0)
        
        # Inverse transform
        dummy = np.zeros((len(predictions_scaled), self.scaler.n_features_in_))
        dummy[:, 0] = predictions_scaled.flatten()
        predictions = self.scaler.inverse_transform(dummy)[:, 0]
        
        return predictions
    
    def evaluate_repair(self, true_values, predicted_values, gap_mask):
        """
        Evaluate repair accuracy on the gaps
        """
        # Only evaluate on positions that had gaps
        true_gap = true_values[gap_mask]
        pred_gap = predicted_values[gap_mask]
        
        if len(true_gap) == 0:
            return None
        
        mae = mean_absolute_error(true_gap, pred_gap)
        rmse = np.sqrt(mean_squared_error(true_gap, pred_gap))
        mape = np.mean(np.abs((true_gap - pred_gap) / (true_gap + 1e-8))) * 100
        r2 = r2_score(true_gap, pred_gap)
        
        return {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'R2': r2,
            'n_gaps': len(true_gap),
            'true_values': true_gap,
            'predicted_values': pred_gap
        }

print("LSTM Data Repairer class defined successfully!")
print("Ready to repair missing data in all scenarios.")

In [ ]:
# Visualize the different scenarios
fig, axes = plt.subplots(4, 1, figsize=(16, 14))

scenarios = [
    (scenario1, "Scenario 1: Random 20% Removal", 0),
    (scenario2, "Scenario 2: Consecutive Blocks (5 × 12h)", 1),
    (scenario3, "Scenario 3: Peak Values Removed (Top 15%)", 2),
    (scenario4, "Scenario 4: Low Values Removed (Bottom 10%)", 3)
]

for scenario_df, title, idx in scenarios:
    # Plot complete data
    axes[idx].plot(scenario_df.index, scenario_df['PM25_complete'], 
                   label='Original Complete Data', linewidth=1.5, alpha=0.7, color='blue')
    
    # Plot data with gaps
    axes[idx].plot(scenario_df.index, scenario_df['PM25_with_gaps'], 
                   label='Data with Gaps', linewidth=1.5, alpha=0.9, color='green')
    
    # Highlight removed points
    removed_data = scenario_df[scenario_df['is_artificial_gap']]
    axes[idx].scatter(removed_data.index, removed_data['PM25_complete'], 
                     color='red', s=30, alpha=0.6, label=f'Removed Points ({len(removed_data)})', 
                     marker='x', zorder=5)
    
    axes[idx].set_title(title, fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('PM2.5 (µg/m³)')
    axes[idx].legend(loc='best')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("\n" + "="*80)
print("MISSING DATA STATISTICS BY SCENARIO")
print("="*80)
for i, (scenario_df, title, _) in enumerate(scenarios, 1):
    n_missing = scenario_df['is_artificial_gap'].sum()
    pct_missing = (n_missing / len(scenario_df)) * 100
    print(f"\nScenario {i}: {n_missing} missing points ({pct_missing:.2f}%)")
print("="*80)

In [ ]:
class DataRemovalSimulator:
    """
    Simulates different patterns of missing data for testing repair capabilities
    """
    
    def __init__(self, df, value_column='PM25_complete'):
        self.df = df.copy()
        self.value_column = value_column
        self.removed_indices = {}
        
    def remove_random(self, percentage=20, seed=42):
        """
        Randomly remove data points
        
        Args:
            percentage: Percentage of data to remove (0-100)
        """
        np.random.seed(seed)
        n_points = len(self.df)
        n_remove = int(n_points * percentage / 100)
        
        # Select random indices
        all_indices = np.arange(n_points)
        remove_indices = np.random.choice(all_indices, n_remove, replace=False)
        
        # Create scenario
        scenario_df = self.df.copy()
        scenario_df['PM25_with_gaps'] = scenario_df[self.value_column].copy()
        scenario_df.loc[scenario_df.index[remove_indices], 'PM25_with_gaps'] = np.nan
        scenario_df['is_artificial_gap'] = False
        scenario_df.loc[scenario_df.index[remove_indices], 'is_artificial_gap'] = True
        
        self.removed_indices['random'] = remove_indices
        
        print(f"Random Removal: Removed {n_remove} points ({percentage}%)")
        return scenario_df
    
    def remove_consecutive_blocks(self, n_blocks=5, block_size=12, seed=42):
        """
        Remove consecutive blocks of data (simulating sensor outages)
        
        Args:
            n_blocks: Number of blocks to remove
            block_size: Size of each block in hours
        """
        np.random.seed(seed)
        n_points = len(self.df)
        
        scenario_df = self.df.copy()
        scenario_df['PM25_with_gaps'] = scenario_df[self.value_column].copy()
        scenario_df['is_artificial_gap'] = False
        
        all_removed = []
        
        for i in range(n_blocks):
            # Select random start position (ensure we don't go out of bounds)
            max_start = n_points - block_size
            if max_start <= 0:
                break
            start_idx = np.random.randint(0, max_start)
            end_idx = start_idx + block_size
            
            # Remove the block
            block_indices = np.arange(start_idx, end_idx)
            scenario_df.loc[scenario_df.index[block_indices], 'PM25_with_gaps'] = np.nan
            scenario_df.loc[scenario_df.index[block_indices], 'is_artificial_gap'] = True
            all_removed.extend(block_indices)
        
        self.removed_indices['consecutive'] = np.array(all_removed)
        
        print(f"Consecutive Block Removal: Removed {len(all_removed)} points in {n_blocks} blocks of {block_size}h")
        return scenario_df
    
    def remove_peak_values(self, top_percentage=10):
        """
        Remove peak values (simulating sensor saturation or errors during high pollution)
        
        Args:
            top_percentage: Percentage of highest values to remove
        """
        n_points = len(self.df)
        n_remove = int(n_points * top_percentage / 100)
        
        # Find indices of top values
        values = self.df[self.value_column].values
        top_indices = np.argsort(values)[-n_remove:]
        
        scenario_df = self.df.copy()
        scenario_df['PM25_with_gaps'] = scenario_df[self.value_column].copy()
        scenario_df.loc[scenario_df.index[top_indices], 'PM25_with_gaps'] = np.nan
        scenario_df['is_artificial_gap'] = False
        scenario_df.loc[scenario_df.index[top_indices], 'is_artificial_gap'] = True
        
        self.removed_indices['peaks'] = top_indices
        
        print(f"Peak Value Removal: Removed {n_remove} highest values ({top_percentage}%)")
        print(f"Removed values range: {values[top_indices].min():.2f} - {values[top_indices].max():.2f} µg/m³")
        return scenario_df
    
    def remove_low_values(self, bottom_percentage=10):
        """
        Remove low values (simulating sensor reading errors during clean air)
        
        Args:
            bottom_percentage: Percentage of lowest values to remove
        """
        n_points = len(self.df)
        n_remove = int(n_points * bottom_percentage / 100)
        
        # Find indices of bottom values
        values = self.df[self.value_column].values
        bottom_indices = np.argsort(values)[:n_remove]
        
        scenario_df = self.df.copy()
        scenario_df['PM25_with_gaps'] = scenario_df[self.value_column].copy()
        scenario_df.loc[scenario_df.index[bottom_indices], 'PM25_with_gaps'] = np.nan
        scenario_df['is_artificial_gap'] = False
        scenario_df.loc[scenario_df.index[bottom_indices], 'is_artificial_gap'] = True
        
        self.removed_indices['low_values'] = bottom_indices
        
        print(f"Low Value Removal: Removed {n_remove} lowest values ({bottom_percentage}%)")
        print(f"Removed values range: {values[bottom_indices].min():.2f} - {values[bottom_indices].max():.2f} µg/m³")
        return scenario_df

# Initialize simulator
simulator = DataRemovalSimulator(df_repair_demo)

# Create different scenarios
print("\n" + "="*80)
print("CREATING MISSING DATA SCENARIOS")
print("="*80 + "\n")

scenario1 = simulator.remove_random(percentage=20, seed=42)
print()
scenario2 = simulator.remove_consecutive_blocks(n_blocks=5, block_size=12, seed=42)
print()
scenario3 = simulator.remove_peak_values(top_percentage=15)
print()
scenario4 = simulator.remove_low_values(bottom_percentage=10)

print("\n" + "="*80)
print("SCENARIOS CREATED")
print("="*80)
print("Scenario 1: Random 20% removal (general missing data)")
print("Scenario 2: 5 consecutive 12-hour blocks (sensor outages)")
print("Scenario 3: Top 15% peak values removed (high pollution measurement failures)")
print("Scenario 4: Bottom 10% low values removed (low pollution reading errors)")
print("="*80)

In [ ]:
# Create a copy of the original complete data for repair demonstration
df_repair_demo = df_featured.copy()

# Remove the original missing data indicator and use filled data
df_repair_demo['PM25_complete'] = df_repair_demo['PM25'].fillna(method='ffill').fillna(method='bfill')

print("="*80)
print("DATA REPAIR DEMONSTRATION SETUP")
print("="*80)
print(f"Total data points: {len(df_repair_demo)}")
print(f"Original complete values: {df_repair_demo['PM25_complete'].notna().sum()}")
print("\nWe will artificially remove data and test the LSTM's ability to repair it.")
print("="*80)

## 5D. DATA REPAIR DEMONSTRATION - Intentional Data Removal and Recovery

This section demonstrates the LSTM model's ability to repair missing data by:
1. **Creating artificial gaps** - Removing data with different patterns (random, consecutive, peak values)
2. **Training on incomplete data** - The model learns from available data
3. **Repairing missing values** - Using learned patterns to fill gaps
4. **Evaluating accuracy** - Comparing repaired values with original ground truth

In [ ]:
# Visual comparison of metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics_names = ['MAE', 'RMSE', 'R² Score']
baseline_values = [baseline_metrics['MAE'], baseline_metrics['RMSE'], baseline_metrics['R2']]
optimized_values = [mae_opt, rmse_opt, r2_opt]

x = np.arange(len(metrics_names))
width = 0.35

# Plot 1: MAE Comparison
axes[0].bar([0], [baseline_metrics['MAE']], width, label='Baseline', color='skyblue', edgecolor='black')
axes[0].bar([0 + width], [mae_opt], width, label='Optimized', color='salmon', edgecolor='black')
axes[0].set_ylabel('MAE (µg/m³)', fontsize=12)
axes[0].set_title('Mean Absolute Error\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[0].set_xticks([width/2])
axes[0].set_xticklabels(['MAE'])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
improvement_text = f"{mae_improvement:+.2f}%"
axes[0].text(width/2, max(baseline_metrics['MAE'], mae_opt) * 1.05, improvement_text, 
             ha='center', fontsize=11, fontweight='bold', color='green' if mae_improvement > 0 else 'red')

# Plot 2: RMSE Comparison
axes[1].bar([0], [baseline_metrics['RMSE']], width, label='Baseline', color='skyblue', edgecolor='black')
axes[1].bar([0 + width], [rmse_opt], width, label='Optimized', color='salmon', edgecolor='black')
axes[1].set_ylabel('RMSE (µg/m³)', fontsize=12)
axes[1].set_title('Root Mean Squared Error\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[1].set_xticks([width/2])
axes[1].set_xticklabels(['RMSE'])
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
improvement_text = f"{rmse_improvement:+.2f}%"
axes[1].text(width/2, max(baseline_metrics['RMSE'], rmse_opt) * 1.05, improvement_text, 
             ha='center', fontsize=11, fontweight='bold', color='green' if rmse_improvement > 0 else 'red')

# Plot 3: R² Comparison
axes[2].bar([0], [baseline_metrics['R2']], width, label='Baseline', color='skyblue', edgecolor='black')
axes[2].bar([0 + width], [r2_opt], width, label='Optimized', color='salmon', edgecolor='black')
axes[2].set_ylabel('R² Score', fontsize=12)
axes[2].set_title('R² Score\n(Higher is Better)', fontsize=13, fontweight='bold')
axes[2].set_xticks([width/2])
axes[2].set_xticklabels(['R²'])
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].set_ylim([0, 1])
r2_diff = r2_opt - baseline_metrics['R2']
improvement_text = f"{r2_diff:+.4f}"
axes[2].text(width/2, max(baseline_metrics['R2'], r2_opt) * 1.05, improvement_text, 
             ha='center', fontsize=11, fontweight='bold', color='green' if r2_diff > 0 else 'red')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("FINAL MODEL RECOMMENDATION")
print("="*80)

if mae_improvement > 5:
    print("✅ RECOMMENDED MODEL: Optimized Model")
    print(f"   The optimized model shows significant improvement ({mae_improvement:.2f}% reduction in MAE)")
    print("   and should be used for production forecasting and data repair.")
elif mae_improvement > 0:
    print("✅ RECOMMENDED MODEL: Optimized Model")
    print(f"   The optimized model shows moderate improvement ({mae_improvement:.2f}% reduction in MAE).")
    print("   Consider using it for production, but monitor performance closely.")
else:
    print("⚠️  RECOMMENDATION: Further tuning needed")
    print("   The optimized model did not significantly outperform the baseline.")
    print("   Consider additional techniques like ensemble methods or hyperparameter tuning.")

print("\n" + "="*80)
print("OPTIMIZATION TECHNIQUES APPLIED")
print("="*80)
print("✓ Bidirectional LSTM layers (2x parameters, processes both directions)")
print("✓ Multi-Head Attention mechanism (4 heads, focuses on relevant time steps)")
print("✓ Batch Normalization (stabilizes training, reduces internal covariate shift)")
print("✓ Residual connections (improves gradient flow)")
print("✓ Huber loss function (more robust to outliers than MSE)")
print("✓ Gradient clipping (prevents exploding gradients)")
print("✓ Extended sequence length (48 hours vs 24 hours)")
print("✓ Advanced features (24 features vs 6 features):")
print("  - Rolling statistics (mean, std, max, min for 3, 6, 12, 24h windows)")
print("  - Lag features (1, 3, 6, 12, 24 hours)")
print("  - Rate of change (1, 3, 6 hours)")
print("  - Cyclical temporal encoding (hour, day of week)")

print("\n" + "="*80)
print("FURTHER IMPROVEMENT SUGGESTIONS")
print("="*80)
print("1. Ensemble Methods:")
print("   - Combine multiple models (LSTM, GRU, CNN-LSTM) for better predictions")
print("   - Use stacking or blending techniques")
print("\n2. Hyperparameter Tuning:")
print("   - Optimize learning rate, batch size, dropout rates")
print("   - Use Keras Tuner or Optuna for automated search")
print("\n3. External Features:")
print("   - Incorporate weather data (temperature, humidity, wind speed/direction)")
print("   - Add nearby station data for spatial context")
print("   - Include calendar features (holidays, special events)")
print("\n4. Advanced Architectures:")
print("   - Try Transformer models (better at capturing long-range dependencies)")
print("   - Implement CNN-LSTM hybrid (extract local patterns + temporal dependencies)")
print("   - Use WaveNet-style dilated convolutions for time series")
print("\n5. Data Augmentation:")
print("   - Generate synthetic data for underrepresented scenarios")
print("   - Use time-series specific augmentation (jittering, warping)")
print("\n6. Cross-validation:")
print("   - Implement time-series cross-validation for more robust evaluation")
print("   - Use walk-forward validation")
print("="*80)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# For fair comparison, use overlapping validation period
# Baseline uses 24h lookback, optimized uses 48h lookback
# So we need to adjust indices appropriately

# Plot 1: Time series comparison (last 100 points of validation set)
plot_points = min(100, len(y_val_true))
x_axis = range(plot_points)

axes[0, 0].plot(x_axis, y_val_true[-plot_points:], label='Actual PM2.5', linewidth=2, alpha=0.8, color='black')
axes[0, 0].plot(x_axis, y_val_pred[-plot_points:], label='Baseline Prediction', linewidth=1.5, alpha=0.7, linestyle='--')
axes[0, 0].set_title('Baseline Model: Predictions vs Actual', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Sample Index')
axes[0, 0].set_ylabel('PM2.5 (µg/m³)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(x_axis, y_val_true_opt[-plot_points:], label='Actual PM2.5', linewidth=2, alpha=0.8, color='black')
axes[0, 1].plot(x_axis, y_val_pred_opt[-plot_points:], label='Optimized Prediction', linewidth=1.5, alpha=0.7, linestyle='--', color='red')
axes[0, 1].set_title('Optimized Model: Predictions vs Actual', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('PM2.5 (µg/m³)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 2: Scatter plots
axes[1, 0].scatter(y_val_true, y_val_pred, alpha=0.4, s=20)
axes[1, 0].plot([y_val_true.min(), y_val_true.max()], 
                [y_val_true.min(), y_val_true.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[1, 0].set_title(f'Baseline: R² = {baseline_metrics["R2"]:.4f}', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Actual PM2.5 (µg/m³)')
axes[1, 0].set_ylabel('Predicted PM2.5 (µg/m³)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].scatter(y_val_true_opt, y_val_pred_opt, alpha=0.4, s=20, color='red')
axes[1, 1].plot([y_val_true_opt.min(), y_val_true_opt.max()], 
                [y_val_true_opt.min(), y_val_true_opt.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[1, 1].set_title(f'Optimized: R² = {r2_opt:.4f}', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Actual PM2.5 (µg/m³)')
axes[1, 1].set_ylabel('Predicted PM2.5 (µg/m³)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot error distribution comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

errors_baseline = y_val_true - y_val_pred
errors_optimized = y_val_true_opt - y_val_pred_opt

axes[0].hist(errors_baseline, bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_title(f'Baseline Error Distribution\nMean Error: {errors_baseline.mean():.4f} µg/m³', 
                  fontsize=13, fontweight='bold')
axes[0].set_xlabel('Prediction Error (µg/m³)')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].hist(errors_optimized, bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_title(f'Optimized Error Distribution\nMean Error: {errors_optimized.mean():.4f} µg/m³', 
                  fontsize=13, fontweight='bold')
axes[1].set_xlabel('Prediction Error (µg/m³)')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Error Distribution Statistics:")
print("="*60)
print(f"{'Metric':<25} {'Baseline':<17} {'Optimized':<17}")
print("-"*60)
print(f"{'Mean Error':<25} {errors_baseline.mean():<17.4f} {errors_optimized.mean():<17.4f}")
print(f"{'Std Dev of Errors':<25} {errors_baseline.std():<17.4f} {errors_optimized.std():<17.4f}")
print(f"{'25th Percentile Error':<25} {np.percentile(np.abs(errors_baseline), 25):<17.4f} {np.percentile(np.abs(errors_optimized), 25):<17.4f}")
print(f"{'75th Percentile Error':<25} {np.percentile(np.abs(errors_baseline), 75):<17.4f} {np.percentile(np.abs(errors_optimized), 75):<17.4f}")
print(f"{'95th Percentile Error':<25} {np.percentile(np.abs(errors_baseline), 95):<17.4f} {np.percentile(np.abs(errors_optimized), 95):<17.4f}")
print("="*60)

In [ ]:
# Make predictions with optimized model
y_pred_optimized_scaled = forecaster_optimized.predict(X_adv)

# Inverse transform
y_pred_optimized = preparator_advanced.inverse_transform_pm25(y_pred_optimized_scaled)
y_true_optimized = preparator_advanced.inverse_transform_pm25(y_adv)

# Evaluate on validation set
val_idx_adv = len(X_train_adv)
y_val_pred_opt = y_pred_optimized[val_idx_adv:val_idx_adv + len(X_val_adv)]
y_val_true_opt = y_true_optimized[val_idx_adv:val_idx_adv + len(X_val_adv)]

# Calculate metrics
mae_opt = mean_absolute_error(y_val_true_opt, y_val_pred_opt)
rmse_opt = np.sqrt(mean_squared_error(y_val_true_opt, y_val_pred_opt))
mse_opt = mean_squared_error(y_val_true_opt, y_val_pred_opt)
r2_opt = r2_score(y_val_true_opt, y_val_pred_opt)

# Additional metrics
mape_opt = np.mean(np.abs((y_val_true_opt - y_val_pred_opt) / (y_val_true_opt + 1e-8))) * 100
max_error_opt = np.max(np.abs(y_val_true_opt - y_val_pred_opt))

print("="*80)
print("OPTIMIZED MODEL PERFORMANCE")
print("="*80)
print(f"MAE:        {mae_opt:.4f} µg/m³")
print(f"RMSE:       {rmse_opt:.4f} µg/m³")
print(f"MSE:        {mse_opt:.4f}")
print(f"R² Score:   {r2_opt:.4f}")
print(f"MAPE:       {mape_opt:.2f}%")
print(f"Max Error:  {max_error_opt:.4f} µg/m³")
print("="*80)

# Calculate improvements
mae_improvement = ((baseline_metrics['MAE'] - mae_opt) / baseline_metrics['MAE']) * 100
rmse_improvement = ((baseline_metrics['RMSE'] - rmse_opt) / baseline_metrics['RMSE']) * 100
r2_improvement = ((r2_opt - baseline_metrics['R2']) / (1 - baseline_metrics['R2'])) * 100

print("\n" + "="*80)
print("PERFORMANCE COMPARISON: Baseline vs Optimized")
print("="*80)
print(f"{'Metric':<15} {'Baseline':<15} {'Optimized':<15} {'Improvement':<15}")
print("-"*80)
print(f"{'MAE (µg/m³)':<15} {baseline_metrics['MAE']:<15.4f} {mae_opt:<15.4f} {mae_improvement:>+.2f}%")
print(f"{'RMSE (µg/m³)':<15} {baseline_metrics['RMSE']:<15.4f} {rmse_opt:<15.4f} {rmse_improvement:>+.2f}%")
print(f"{'R² Score':<15} {baseline_metrics['R2']:<15.4f} {r2_opt:<15.4f} {r2_improvement:>+.2f}%")
print(f"{'MSE':<15} {baseline_metrics['MSE']:<15.4f} {mse_opt:<15.4f}")
print("="*80)

if mae_improvement > 0:
    print(f"\n✅ SUCCESS: The optimized model achieved {mae_improvement:.2f}% improvement in MAE!")
else:
    print(f"\n⚠️  The optimized model did not improve MAE, but may perform better on other metrics.")

print(f"\nKey Insights:")
print(f"- The optimized model reduced prediction error by {mae_improvement:.2f}%")
print(f"- RMSE improved by {rmse_improvement:.2f}%, indicating better handling of large errors")
print(f"- R² score of {r2_opt:.4f} means the model explains {r2_opt*100:.2f}% of variance")

## 5C. Model Comparison: Baseline vs Optimized

Compare the performance of both models to quantify improvement.

In [ ]:
# Train the optimized model
history_optimized = forecaster_optimized.train(
    X_train_adv, y_train_adv, 
    X_val_adv, y_val_adv, 
    epochs=150, 
    batch_size=32
)

# Plot training history
forecaster_optimized.plot_training_history()

In [ ]:
# Import additional layers for optimized model
from tensorflow.keras.layers import (
    Bidirectional, BatchNormalization, Attention, 
    Concatenate, MultiHeadAttention, LayerNormalization
)

class OptimizedLSTMForecaster:
    """
    Advanced LSTM model with Bidirectional layers, Attention mechanism, and Batch Normalization
    """
    
    def __init__(self, sequence_length=48, n_features=18):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.model = None
        self.history = None
    
    def build_attention_model(self):
        """
        Build optimized architecture with:
        - Bidirectional LSTM layers
        - Multi-Head Attention mechanism
        - Batch Normalization
        - Residual connections
        """
        inputs = Input(shape=(self.sequence_length, self.n_features), name='input')
        
        # First Bidirectional LSTM block
        x = Bidirectional(LSTM(128, return_sequences=True, activation='tanh'))(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        # Second Bidirectional LSTM block
        x = Bidirectional(LSTM(64, return_sequences=True, activation='tanh'))(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        # Multi-Head Attention layer
        attention_output = MultiHeadAttention(
            num_heads=4,
            key_dim=32,
            dropout=0.1
        )(x, x)
        
        # Add & Normalize (Residual connection)
        x = LayerNormalization()(x + attention_output)
        
        # Third Bidirectional LSTM block
        x = Bidirectional(LSTM(32, return_sequences=False, activation='tanh'))(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        # Dense layers
        x = Dense(64, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.1)(x)
        
        outputs = Dense(1, name='output')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        # Use Adam optimizer with gradient clipping
        optimizer = keras.optimizers.Adam(
            learning_rate=0.001,
            clipnorm=1.0  # Gradient clipping for stability
        )
        
        model.compile(
            optimizer=optimizer,
            loss='huber',  # Huber loss is more robust to outliers than MSE
            metrics=['mae', 'mse']
        )
        
        self.model = model
        return model
    
    def train(self, X_train, y_train, X_val, y_val, epochs=150, batch_size=32):
        """
        Train with advanced callbacks and regularization
        """
        # Callbacks
        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        )
        
        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=8,
            min_lr=0.000001,
            verbose=1
        )
        
        # Model checkpoint
        checkpoint = callbacks.ModelCheckpoint(
            'best_optimized_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=0
        )
        
        print("Training OPTIMIZED LSTM model with Attention...")
        print("="*60)
        self.history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, reduce_lr, checkpoint],
            verbose=1
        )
        
        return self.history
    
    def predict(self, X):
        """
        Make predictions
        """
        return self.model.predict(X, verbose=0)
    
    def plot_training_history(self):
        """
        Visualize training progress
        """
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss
        axes[0].plot(self.history.history['loss'], label='Training Loss', linewidth=2)
        axes[0].plot(self.history.history['val_loss'], label='Validation Loss', linewidth=2)
        axes[0].set_title('Optimized Model Loss During Training', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss (Huber)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # MAE
        axes[1].plot(self.history.history['mae'], label='Training MAE', linewidth=2)
        axes[1].plot(self.history.history['val_mae'], label='Validation MAE', linewidth=2)
        axes[1].set_title('Optimized Model MAE During Training', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('MAE')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Build optimized model
print("Building OPTIMIZED model architecture...")
print("="*80)
forecaster_optimized = OptimizedLSTMForecaster(
    sequence_length=preparator_advanced.sequence_length, 
    n_features=len(advanced_feature_cols)
)
forecaster_optimized.build_attention_model()

print("\nOptimized Model Architecture:")
print("="*80)
forecaster_optimized.model.summary()
print("\nKey improvements:")
print("✓ Bidirectional LSTM (processes sequences forward and backward)")
print("✓ Multi-Head Attention (focuses on relevant time steps)")
print("✓ Batch Normalization (stabilizes training)")
print("✓ Residual connections (improves gradient flow)")
print("✓ Huber loss (robust to outliers)")
print("✓ Gradient clipping (prevents exploding gradients)")
print("✓ Longer sequence length (48 hours vs 24 hours)")
print("✓ Advanced features (rolling stats, lags, rate of change)")
print("="*80)

In [ ]:
# Create sequences with advanced features
X_adv, y_adv, masks_adv = preparator_advanced.create_sequences(
    df_featured_advanced, 
    target_col='PM25',
    feature_cols=advanced_feature_cols
)

print(f"Advanced sequence shapes:")
print(f"X shape: {X_adv.shape} (samples, sequence_length={preparator_advanced.sequence_length}, features={len(advanced_feature_cols)})")
print(f"y shape: {y_adv.shape}")

# Split into train and validation
non_missing_idx_adv = ~masks_adv
X_clean_adv = X_adv[non_missing_idx_adv]
y_clean_adv = y_adv[non_missing_idx_adv]

X_train_adv, X_val_adv, y_train_adv, y_val_adv = train_test_split(
    X_clean_adv, y_clean_adv, test_size=0.2, random_state=42, shuffle=False
)

print(f"\nAdvanced data splits:")
print(f"Training set: {len(X_train_adv)} samples")
print(f"Validation set: {len(X_val_adv)} samples")

In [ ]:
# Advanced Feature Engineering
class AdvancedDataPreparator(DataPreparator):
    """
    Enhanced data preparation with advanced features
    """
    
    def __init__(self, sequence_length=48):  # Increased to 48 hours (2 days)
        super().__init__(sequence_length)
        
    def create_advanced_features(self, df):
        """
        Create additional statistical and lag features
        """
        df = super().create_features(df)
        
        # Fill PM25 temporarily for feature calculation
        pm25_filled = df['PM25'].fillna(method='ffill').fillna(method='bfill')
        
        # Rolling statistics (3, 6, 12, 24 hour windows)
        for window in [3, 6, 12, 24]:
            df[f'PM25_rolling_mean_{window}h'] = pm25_filled.rolling(window=window, min_periods=1).mean()
            df[f'PM25_rolling_std_{window}h'] = pm25_filled.rolling(window=window, min_periods=1).std().fillna(0)
            df[f'PM25_rolling_max_{window}h'] = pm25_filled.rolling(window=window, min_periods=1).max()
            df[f'PM25_rolling_min_{window}h'] = pm25_filled.rolling(window=window, min_periods=1).min()
        
        # Lag features (1, 3, 6, 12, 24 hours ago)
        for lag in [1, 3, 6, 12, 24]:
            df[f'PM25_lag_{lag}h'] = pm25_filled.shift(lag)
        
        # Rate of change
        df['PM25_change_1h'] = pm25_filled.diff(1).fillna(0)
        df['PM25_change_3h'] = pm25_filled.diff(3).fillna(0)
        df['PM25_change_6h'] = pm25_filled.diff(6).fillna(0)
        
        # Fill any NaN values created by feature engineering
        df = df.fillna(method='bfill').fillna(method='ffill')
        
        return df

# Create advanced features
preparator_advanced = AdvancedDataPreparator(sequence_length=48)
df_featured_advanced = preparator_advanced.create_advanced_features(df)

print("Advanced Features Created:")
print(f"Total features: {len(df_featured_advanced.columns)}")
print("\nNew feature categories:")
print("- Rolling statistics: 16 features (mean, std, max, min for 4 windows)")
print("- Lag features: 5 features (1, 3, 6, 12, 24 hours)")
print("- Rate of change: 3 features (1, 3, 6 hours)")

# Select important features for modeling
advanced_feature_cols = [
    'PM25', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend',
    'PM25_rolling_mean_3h', 'PM25_rolling_mean_6h', 'PM25_rolling_mean_12h', 'PM25_rolling_mean_24h',
    'PM25_rolling_std_3h', 'PM25_rolling_std_12h',
    'PM25_lag_1h', 'PM25_lag_3h', 'PM25_lag_6h', 'PM25_lag_24h',
    'PM25_change_1h', 'PM25_change_3h'
]

print(f"\nSelected features for modeling: {len(advanced_feature_cols)}")
print(advanced_feature_cols)

## 5B. MODEL OPTIMIZATION - Advanced Architectures for Improved Accuracy

Now let's implement advanced techniques to improve model accuracy:
- **Bidirectional LSTM**: Process sequences in both directions
- **Attention Mechanism**: Focus on relevant time steps
- **Batch Normalization**: Stabilize training
- **Advanced Feature Engineering**: Rolling statistics and lag features
- **Longer Sequence Length**: Use 48-72 hours of historical data

## 6. Autoencoder for Anomaly Detection

Build an autoencoder to detect anomalous patterns in the data.

In [ ]:
class AnomalyDetector:
    """
    LSTM Autoencoder for anomaly detection in time series data
    """
    
    def __init__(self, sequence_length=24, n_features=6):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.model = None
        self.threshold = None
        
    def build_model(self):
        """
        Build LSTM Autoencoder architecture
        """
        # Encoder
        inputs = Input(shape=(self.sequence_length, self.n_features))
        encoded = LSTM(64, activation='relu', return_sequences=True)(inputs)
        encoded = LSTM(32, activation='relu', return_sequences=False)(encoded)
        encoded = Dropout(0.2)(encoded)
        
        # Bottleneck
        bottleneck = Dense(16, activation='relu')(encoded)
        
        # Decoder
        decoded = RepeatVector(self.sequence_length)(bottleneck)
        decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
        decoded = LSTM(64, activation='relu', return_sequences=True)(decoded)
        decoded = TimeDistributed(Dense(self.n_features))(decoded)
        
        # Autoencoder model
        autoencoder = Model(inputs=inputs, outputs=decoded)
        autoencoder.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse'
        )
        
        self.model = autoencoder
        return autoencoder
    
    def train(self, X_train, X_val, epochs=50, batch_size=32):
        """
        Train the autoencoder on normal data
        """
        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        print("Training Autoencoder for Anomaly Detection...")
        history = self.model.fit(
            X_train, X_train,  # Autoencoder reconstructs its input
            validation_data=(X_val, X_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=1
        )
        
        return history
    
    def calculate_reconstruction_error(self, X):
        """
        Calculate reconstruction error for each sequence
        """
        reconstructed = self.model.predict(X, verbose=0)
        mse = np.mean(np.square(X - reconstructed), axis=(1, 2))
        return mse
    
    def set_threshold(self, X_train, percentile=95):
        """
        Set anomaly threshold based on training data reconstruction error
        """
        train_errors = self.calculate_reconstruction_error(X_train)
        self.threshold = np.percentile(train_errors, percentile)
        print(f"Anomaly threshold set at {percentile}th percentile: {self.threshold:.6f}")
        return self.threshold
    
    def detect_anomalies(self, X):
        """
        Detect anomalies based on reconstruction error
        """
        errors = self.calculate_reconstruction_error(X)
        anomalies = errors > self.threshold
        return anomalies, errors

# Build and train anomaly detector
detector = AnomalyDetector(sequence_length=24, n_features=X.shape[2])
detector.build_model()

print("Autoencoder Architecture:")
detector.model.summary()

In [ ]:
# Train autoencoder on clean data
history_ae = detector.train(X_train, X_val, epochs=50, batch_size=32)

# Set anomaly threshold
detector.set_threshold(X_train, percentile=95)

# Detect anomalies in all data
anomalies, reconstruction_errors = detector.detect_anomalies(X)

print(f"\nAnomaly Detection Results:")
print(f"Total sequences: {len(anomalies)}")
print(f"Anomalies detected: {anomalies.sum()} ({anomalies.sum()/len(anomalies)*100:.2f}%)")

## 7. Anomaly Analysis and Visualization

In [ ]:
# Add anomaly information to repaired dataframe
df_repaired['reconstruction_error'] = np.nan
df_repaired['is_anomaly'] = False

# Align anomalies with dataframe
anomaly_indices = df_repaired.index[preparator.sequence_length:preparator.sequence_length + len(anomalies)]
df_repaired.loc[anomaly_indices, 'reconstruction_error'] = reconstruction_errors
df_repaired.loc[anomaly_indices, 'is_anomaly'] = anomalies

# Analyze anomalies
anomaly_data = df_repaired[df_repaired['is_anomaly']]

print("Anomaly Analysis:")
print(f"\nTotal anomalous time points: {len(anomaly_data)}")
if len(anomaly_data) > 0:
    print(f"Average PM2.5 during anomalies: {anomaly_data['PM25_repaired'].mean():.2f} µg/m³")
    print(f"Max PM2.5 during anomalies: {anomaly_data['PM25_repaired'].max():.2f} µg/m³")
    print(f"Min PM2.5 during anomalies: {anomaly_data['PM25_repaired'].min():.2f} µg/m³")
    
    # Temporal pattern analysis
    print(f"\nTemporal Patterns of Anomalies:")
    print(f"Most common hour: {anomaly_data['hour'].mode().values[0] if len(anomaly_data['hour'].mode()) > 0 else 'N/A'}")
    print(f"Most common day of week: {anomaly_data['day_of_week'].mode().values[0] if len(anomaly_data['day_of_week'].mode()) > 0 else 'N/A'} (0=Monday, 6=Sunday)")
    print(f"Weekend anomalies: {(anomaly_data['is_weekend'] == 1).sum()} ({(anomaly_data['is_weekend'] == 1).sum()/len(anomaly_data)*100:.1f}%)")

# Visualize anomalies
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Plot 1: Time series with anomalies highlighted
axes[0].plot(df_repaired.index, df_repaired['PM25_repaired'], 
             label='PM2.5 (Repaired)', linewidth=1.5, color='blue', alpha=0.7)
if len(anomaly_data) > 0:
    axes[0].scatter(anomaly_data.index, anomaly_data['PM25_repaired'], 
                    color='red', s=100, alpha=0.8, label=f'Anomalies ({len(anomaly_data)})', 
                    marker='X', zorder=5, edgecolors='darkred', linewidths=1.5)
axes[0].set_title('PM2.5 Time Series with Detected Anomalies', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Reconstruction error
axes[1].plot(df_repaired.index[preparator.sequence_length:], reconstruction_errors, 
             label='Reconstruction Error', linewidth=1.5, color='purple', alpha=0.7)
axes[1].axhline(y=detector.threshold, color='red', linestyle='--', linewidth=2, 
                label=f'Anomaly Threshold ({detector.threshold:.6f})')
axes[1].set_title('Autoencoder Reconstruction Error', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Reconstruction Error (MSE)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

# Plot 3: Hourly distribution of anomalies
if len(anomaly_data) > 0:
    hourly_anomalies = anomaly_data.groupby('hour').size()
    all_hours = pd.Series(0, index=range(24))
    all_hours.update(hourly_anomalies)
    axes[2].bar(all_hours.index, all_hours.values, color='orangered', alpha=0.7, edgecolor='black')
    axes[2].set_title('Hourly Distribution of Anomalies', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Hour of Day')
    axes[2].set_ylabel('Number of Anomalies')
    axes[2].grid(True, alpha=0.3, axis='y')
    axes[2].set_xticks(range(24))

plt.tight_layout()
plt.show()

## 8. Potential Causes of Anomalies

Analyze and categorize detected anomalies.

In [ ]:
def analyze_anomaly_causes(df_repaired, anomaly_data):
    """
    Analyze potential causes of detected anomalies
    """
    if len(anomaly_data) == 0:
        print("No anomalies detected to analyze.")
        return
    
    print("="*80)
    print("ANOMALY CAUSE ANALYSIS")
    print("="*80)
    
    # Calculate statistics
    overall_mean = df_repaired['PM25_repaired'].mean()
    overall_std = df_repaired['PM25_repaired'].std()
    
    # Categorize anomalies
    categories = {
        'Extreme High Values': [],
        'Extreme Low Values': [],
        'Sudden Spikes': [],
        'Sudden Drops': [],
        'Unusual Patterns': []
    }
    
    for idx in anomaly_data.index:
        value = df_repaired.loc[idx, 'PM25_repaired']
        
        # Check for extreme values (beyond 2.5 standard deviations)
        if value > overall_mean + 2.5 * overall_std:
            categories['Extreme High Values'].append((idx, value))
        elif value < overall_mean - 2.5 * overall_std and value >= 0:
            categories['Extreme Low Values'].append((idx, value))
        
        # Check for sudden changes
        try:
            idx_pos = df_repaired.index.get_loc(idx)
            if idx_pos > 0:
                prev_value = df_repaired.iloc[idx_pos - 1]['PM25_repaired']
                change = value - prev_value
                
                if abs(change) > 2 * overall_std:
                    if change > 0:
                        categories['Sudden Spikes'].append((idx, value, change))
                    else:
                        categories['Sudden Drops'].append((idx, value, change))
                else:
                    categories['Unusual Patterns'].append((idx, value))
        except:
            categories['Unusual Patterns'].append((idx, value))
    
    # Report findings
    print(f"\n1. EXTREME HIGH VALUES ({len(categories['Extreme High Values'])} detected)")
    print("   Potential causes: Industrial emissions, traffic congestion, wildfires, burning activities")
    if len(categories['Extreme High Values']) > 0:
        for idx, value in categories['Extreme High Values'][:5]:  # Show first 5
            print(f"   - {idx}: {value:.2f} µg/m³ (threshold: {overall_mean + 2.5 * overall_std:.2f})")
    
    print(f"\n2. EXTREME LOW VALUES ({len(categories['Extreme Low Values'])} detected)")
    print("   Potential causes: Heavy rain, strong winds, sensor malfunction, data transmission errors")
    if len(categories['Extreme Low Values']) > 0:
        for idx, value in categories['Extreme Low Values'][:5]:
            print(f"   - {idx}: {value:.2f} µg/m³ (threshold: {overall_mean - 2.5 * overall_std:.2f})")
    
    print(f"\n3. SUDDEN SPIKES ({len(categories['Sudden Spikes'])} detected)")
    print("   Potential causes: Nearby construction, vehicle emissions, local burning, sensor errors")
    if len(categories['Sudden Spikes']) > 0:
        for idx, value, change in categories['Sudden Spikes'][:5]:
            print(f"   - {idx}: {value:.2f} µg/m³ (change: +{change:.2f})")
    
    print(f"\n4. SUDDEN DROPS ({len(categories['Sudden Drops'])} detected)")
    print("   Potential causes: Weather changes, wind shifts, rain events, sensor recalibration")
    if len(categories['Sudden Drops']) > 0:
        for idx, value, change in categories['Sudden Drops'][:5]:
            print(f"   - {idx}: {value:.2f} µg/m³ (change: {change:.2f})")
    
    print(f"\n5. UNUSUAL PATTERNS ({len(categories['Unusual Patterns'])} detected)")
    print("   Potential causes: Data quality issues, sensor drift, unusual meteorological conditions")
    if len(categories['Unusual Patterns']) > 0:
        for idx, value in categories['Unusual Patterns'][:5]:
            print(f"   - {idx}: {value:.2f} µg/m³")
    
    print("\n" + "="*80)
    print("RECOMMENDATIONS")
    print("="*80)
    print("1. Verify sensor calibration and maintenance schedule")
    print("2. Cross-reference anomalies with weather data (rain, wind speed/direction)")
    print("3. Check for nearby activities (construction, traffic events, agricultural burning)")
    print("4. Review data transmission logs for communication errors")
    print("5. Consider implementing real-time alerts for extreme values")
    print("="*80)

# Perform cause analysis
analyze_anomaly_causes(df_repaired, anomaly_data)

## 9. Future Forecasting

Use the trained LSTM model to forecast PM2.5 values for the next 24-48 hours.

In [ ]:
def forecast_future(df_featured, forecaster, preparator, hours_ahead=48):
    """
    Forecast PM2.5 values for future hours
    """
    print(f"Forecasting PM2.5 for next {hours_ahead} hours...")
    
    # Get the last sequence from the data
    last_sequence = df_featured.iloc[-preparator.sequence_length:].copy()
    
    # Prepare features
    feature_cols = ['PM25_repaired', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend']
    
    forecasts = []
    forecast_dates = []
    
    current_sequence = last_sequence[feature_cols].values
    current_time = df_featured.index[-1]
    
    for i in range(hours_ahead):
        # Scale the sequence
        scaled_sequence = preparator.scaler.transform(current_sequence)
        
        # Reshape for model input
        X_forecast = scaled_sequence.reshape(1, preparator.sequence_length, len(feature_cols))
        
        # Predict next value
        pred_scaled = forecaster.predict(X_forecast)
        
        # Inverse transform
        pred_value = preparator.inverse_transform_pm25(pred_scaled)[0]
        
        # Update time
        current_time = current_time + pd.Timedelta(hours=1)
        
        # Create features for the predicted time
        hour = current_time.hour
        day_of_week = current_time.dayofweek
        is_weekend = 1 if day_of_week >= 5 else 0
        
        hour_sin = np.sin(2 * np.pi * hour / 24)
        hour_cos = np.cos(2 * np.pi * hour / 24)
        day_sin = np.sin(2 * np.pi * day_of_week / 7)
        day_cos = np.cos(2 * np.pi * day_of_week / 7)
        
        # Create new row with prediction
        new_row = np.array([pred_value, hour_sin, hour_cos, day_sin, day_cos, is_weekend])
        
        # Update sequence (remove first row, add new row)
        current_sequence = np.vstack([current_sequence[1:], new_row])
        
        forecasts.append(pred_value)
        forecast_dates.append(current_time)
    
    # Create forecast dataframe
    forecast_df = pd.DataFrame({
        'datetime': forecast_dates,
        'PM25_forecast': forecasts
    })
    forecast_df.set_index('datetime', inplace=True)
    
    return forecast_df

# Generate forecast
forecast_df = forecast_future(df_repaired, forecaster, preparator, hours_ahead=48)

print("\nForecast Summary:")
print(forecast_df.describe())

# Visualize forecast
fig, ax = plt.subplots(figsize=(15, 6))

# Plot historical data (last 7 days)
lookback_hours = 7 * 24
historical = df_repaired.iloc[-lookback_hours:]

ax.plot(historical.index, historical['PM25_repaired'], 
        label='Historical PM2.5', linewidth=2, color='blue', alpha=0.7)
ax.plot(forecast_df.index, forecast_df['PM25_forecast'], 
        label='Forecasted PM2.5', linewidth=2, color='red', alpha=0.7, linestyle='--')

# Add vertical line at forecast start
ax.axvline(x=df_repaired.index[-1], color='green', linestyle=':', linewidth=2, label='Forecast Start')

# Add PM2.5 threshold lines
ax.axhline(y=37.5, color='orange', linestyle='--', alpha=0.5, linewidth=1, label='Moderate (37.5 µg/m³)')
ax.axhline(y=50, color='red', linestyle='--', alpha=0.5, linewidth=1, label='Unhealthy (50 µg/m³)')

ax.set_title('PM2.5 Forecast (Next 48 Hours)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5 (µg/m³)')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print forecast warnings
print("\n" + "="*80)
print("FORECAST ALERTS")
print("="*80)

high_risk_hours = forecast_df[forecast_df['PM25_forecast'] > 50]
moderate_risk_hours = forecast_df[(forecast_df['PM25_forecast'] > 37.5) & (forecast_df['PM25_forecast'] <= 50)]

if len(high_risk_hours) > 0:
    print(f"\n⚠️  HIGH RISK: {len(high_risk_hours)} hours forecasted with PM2.5 > 50 µg/m³")
    print("   Action: Avoid outdoor activities, use air purifiers, wear N95 masks if going outside")
    
if len(moderate_risk_hours) > 0:
    print(f"\n⚠️  MODERATE RISK: {len(moderate_risk_hours)} hours forecasted with PM2.5 > 37.5 µg/m³")
    print("   Action: Sensitive groups should limit outdoor exposure")
    
if len(high_risk_hours) == 0 and len(moderate_risk_hours) == 0:
    print("\n✅ GOOD: Air quality forecasted to remain at acceptable levels")

print("\nAverage forecasted PM2.5: {:.2f} µg/m³".format(forecast_df['PM25_forecast'].mean()))
print("Peak forecasted PM2.5: {:.2f} µg/m³ at {}".format(
    forecast_df['PM25_forecast'].max(),
    forecast_df['PM25_forecast'].idxmax()
))
print("="*80)

## 10. Summary Report

Generate a comprehensive summary of all analyses.

In [ ]:
print("="*80)
print("DEEP LEARNING AIR QUALITY ANALYSIS - COMPREHENSIVE REPORT")
print("="*80)

print("\n📊 DATA COLLECTION")
print("-" * 80)
print(f"Station ID: {fetcher.station_id}")
print(f"Date Range: {start_date} to {end_date}")
print(f"Total Data Points: {len(df_repaired)}")
print(f"Original Missing Values: {df['is_missing'].sum()} ({df['is_missing'].sum()/len(df)*100:.2f}%)")

print("\n🤖 MODEL PERFORMANCE")
print("-" * 80)
print("LSTM Forecasting Model:")
print(f"  - Mean Absolute Error (MAE): {mae:.3f} µg/m³")
print(f"  - Root Mean Squared Error (RMSE): {rmse:.3f} µg/m³")
print(f"  - R² Score: {r2:.3f}")
print(f"  - Missing Values Repaired: {repaired_count}")

print("\n🔍 ANOMALY DETECTION")
print("-" * 80)
print(f"Autoencoder Anomaly Threshold: {detector.threshold:.6f}")
print(f"Total Anomalies Detected: {anomalies.sum()} ({anomalies.sum()/len(anomalies)*100:.2f}%)")
if len(anomaly_data) > 0:
    print(f"Average PM2.5 during Anomalies: {anomaly_data['PM25_repaired'].mean():.2f} µg/m³")
    print(f"Peak Anomaly Value: {anomaly_data['PM25_repaired'].max():.2f} µg/m³")

print("\n📈 STATISTICAL SUMMARY")
print("-" * 80)
print(f"Mean PM2.5: {df_repaired['PM25_repaired'].mean():.2f} µg/m³")
print(f"Median PM2.5: {df_repaired['PM25_repaired'].median():.2f} µg/m³")
print(f"Std Dev: {df_repaired['PM25_repaired'].std():.2f} µg/m³")
print(f"Min PM2.5: {df_repaired['PM25_repaired'].min():.2f} µg/m³")
print(f"Max PM2.5: {df_repaired['PM25_repaired'].max():.2f} µg/m³")

# Air quality classification
good_hours = (df_repaired['PM25_repaired'] <= 37.5).sum()
moderate_hours = ((df_repaired['PM25_repaired'] > 37.5) & (df_repaired['PM25_repaired'] <= 50)).sum()
unhealthy_hours = (df_repaired['PM25_repaired'] > 50).sum()

print("\n🌍 AIR QUALITY CLASSIFICATION")
print("-" * 80)
print(f"Good (≤37.5 µg/m³): {good_hours} hours ({good_hours/len(df_repaired)*100:.1f}%)")
print(f"Moderate (37.5-50 µg/m³): {moderate_hours} hours ({moderate_hours/len(df_repaired)*100:.1f}%)")
print(f"Unhealthy (>50 µg/m³): {unhealthy_hours} hours ({unhealthy_hours/len(df_repaired)*100:.1f}%)")

print("\n🔮 FORECAST")
print("-" * 80)
print(f"Forecast Period: Next 48 hours")
print(f"Average Forecasted PM2.5: {forecast_df['PM25_forecast'].mean():.2f} µg/m³")
print(f"Peak Forecasted PM2.5: {forecast_df['PM25_forecast'].max():.2f} µg/m³")
print(f"Minimum Forecasted PM2.5: {forecast_df['PM25_forecast'].min():.2f} µg/m³")

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE")
print("="*80)

## 11. Export Results

Save the processed data and models for future use.

In [ ]:
# Save repaired data
output_file = 'air_quality_repaired_data.csv'
df_repaired.to_csv(output_file)
print(f"Repaired data saved to: {output_file}")

# Save forecast
forecast_file = 'air_quality_forecast_48h.csv'
forecast_df.to_csv(forecast_file)
print(f"Forecast data saved to: {forecast_file}")

# Save models (optional)
# forecaster.model.save('lstm_forecaster_model.h5')
# detector.model.save('autoencoder_anomaly_detector.h5')
# print("Models saved successfully")

print("\nAll results exported successfully!")